In [ ]:
import pandas as pd
import os
import plotly.express as px
import plotly.colors as plotcolor
import plotly.graph_objects as go
import datapane as dp
from plotly.subplots import make_subplots
from datetime import datetime 
from logger import log
import copy
pd.options.mode.chained_assignment = None  # default='warn'
log("starting mevboost-pics chart generation")

GITFOLDER = "../mevboost.pics-pages/"

BLACK = "rgb(15, 20, 25)"
BLACK_ALPHA = "rgba(15, 20, 25, {})"
#px.colors.qualitative.Plotly = ['#636EFA', "#EF553B", "#00CC96", "#AB63FA", "#FFA15A", "#19D3F3", 
#                                "#FF97FF", "#7f7f7f", "#B6E880", "#FF6692", "#636EFA"]

WHITE = "rgba(255, 255, 255, 0.1)"


colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", 
          "#7f7f7f", "#bcbd22", "#17becf", "#b1d7e5", "#e377c2"]
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",
          "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#b1d7e5"]
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",          
          "#8c564b", "#e377c2", "#7f7f7f", "#e6e6e6", "#b1d7e5"]

colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",          
          "#8c564b", "#e377c2", "#7f7f7f", "#b1d7e5", "#f6ff42", "#02e382", "#e6e6e6", "#ffbb78"]

colors_val = ['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', 
              '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f', '#a6cee3', 
              '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00']

colors_builder = ['#1f77b4', '#F44336', '#E91E63', '#9C27B0', '#673AB7', '#3F51B5',    
                  '#2196F3', '#03A9F4', '#00BCD4', '#009688', '#4CAF50',    
                  '#8BC34A', '#CDDC39', '#FFEB3B', '#FFC107', '#FF9800',    
                  '#FF5722', '#795548', '#9E9E9E', '#607D8B', '#1abc9c',    
                  '#2ecc71', '#3498db', '#34495e', '#e74c3c', '#f1c40f',    
                  '#16a085', '#8e44ad', '#e67e22', "#1f77b4"]



px.colors.qualitative.Plotly = colors + px.colors.qualitative.Plotly
px.colors.DEFAULT_PLOTLY_COLORS = px.colors.qualitative.Plotly

# {'Flashbots': '#1f77b4',
#  'Bloxroute (max profit)': '#ff7f0e',
#  'Other/None': '#2ca02c',
#  'Blocknative': '#d62728',
#  'Bloxroute (regulated)': '#9467bd',
#  'Eden': '#8c564b',
#  'Agnostic Gnosis': '#e377c2',
#  'Manifold': '#7f7f7f',
#  'Bloxroute (ethical)': '#b1d7e5',
#  'Ultrasound': '#f6ff42',
#  'Relayooor': '#02e382',
#  'Aestus': '#e6e6e6'}


def get_timestamp_of_slot(slot):
    date = 1654824023 + (slot-int(4e6))*12
    date = datetime.strftime(datetime.utcfromtimestamp(date), "%Y-%m-%d, %I:%M %p") + " +UTC"
    return date

def get_second_in_slot(slot, timestamp):
    slotbegin = 1654824023 + (slot-1-int(4e6))*12
    return timestamp-slotbegin

KEY_DIR = "key/"
CREDENTIALS = "google-creds.json"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = KEY_DIR \
    + CREDENTIALS or input("No Google API credendials file provided." 
    + "Please specify path now:\n")

QUERY = """
SELECT {}
FROM {}
{}
"""

def build_query(select, dataset, appendix=""):
    return QUERY.format(select, dataset, appendix)

def get_last_known_slot():
    df = pd.read_gbq("""SELECT max(slot) slot FROM `ethereum-data-nero.eth.mevboost_empty_filled` 
                WHERE DATE(date) >  DATE_SUB(DATE_TRUNC(current_date(), WEEK(MONDAY)), INTERVAL 1 WEEK)
                LIMIT 1""")
    return int(df["slot"])

def hextorgba(h, alpha=0.20):
    return "rgba"+str(tuple(int(h[i:i+2], 16) for i in (0, 2, 4)) + (alpha,))

def brighter_color(hex_color, brightness):
    """
    Takes a hex color value and increases its brightness by a given amount.
    Returns the new hex color value as a string.

    Args:
    hex_color (str): a hex color code in the format #RRGGBB or #RGB
    brightness (float): the amount of brightness to add (between 0 and 1)

    Returns:
    str: the new hex color code in the format #RRGGBB or #RGB
    """
    # Remove the '#' character from the hex color code
    hex_color = hex_color.lstrip('#')

    # If the hex code is in the short format (e.g. #ABC), expand it to the long format (e.g. #AABBCC)
    if len(hex_color) == 3:
        hex_color = ''.join([c * 2 for c in hex_color])

    # Convert the hex code to an RGB tuple
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    # Calculate the new RGB values by adding the brightness to each component
    new_rgb = tuple(min(int(c + 255 * brightness), 255) for c in rgb)

    # Convert the new RGB values back to a hex code
    new_hex = '#{:02X}{:02X}{:02X}'.format(*new_rgb)

    return new_hex


def clean_xml(df_mapping):
    try:
        def valid_xml_char_ordinal(c):
            codepoint = ord(c)
            # conditions ordered by presumed frequency
            return (
                0x20 <= codepoint <= 0xD7FF or
                codepoint in (0x9, 0xA, 0xD) or
                0xE000 <= codepoint <= 0xFFFD or
                0x10000 <= codepoint <= 0x10FFFF
                )
        return ''.join(c for c in df_mapping.to_html(index=False) if valid_xml_char_ordinal(c))
    except:
        return ""

try:
    get_ipython().__class__.__name__
    JUPYTER = True
except:
    JUPYTER = False
#JUPYTER = False
    
print(f"Running in Jupyter Notebook: {JUPYTER}")

LAST_SLOT = get_last_known_slot()
LAST_TS = get_timestamp_of_slot(LAST_SLOT)

In [ ]:
#from IPython.display import HTML
#
##colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",          "#8c564b", "#e377c2", "#7f7f7f", "#e6e6e6", "#b1d7e5"]
#
## create a string with a span element for each color
#colors_html = "".join(f"<span style='display:inline-block;width:40px;height:20px;background-color:{color}'></span>" for color in colors)
#
## display the colors using HTML
#HTML(colors_html)
#

In [ ]:
def create_avg_proposer_payment_relay_boxes():  
    print("start create_avg_proposer_payment_relay_boxes")
    df = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_relay_boxplot_24h`').replace("ultrasound", "ultra Sound")
    df["relay"] = df["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    df2 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_relay_boxplot_48h`').replace("ultrasound", "ultra Sound")
    df2["relay"] = df2["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    df3 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_relay_boxplot_1w`').replace("ultrasound", "ultra Sound")
    df3["relay"] = df3["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    df4 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_relay_boxplot_1m`').replace("ultrasound", "ultra Sound")
    df4["relay"] = df4["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    df5 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_relay_boxplot_6m`').replace("ultrasound", "ultra Sound")
    df5["relay"] = df5["relay"].apply(lambda x: x[0].upper()+x[1:])
    
    maxy24 = max(df["percentile75"])*1.1
    maxy48 = max(df2["percentile75"])*1.1
    maxy1w = max(df3["percentile75"])*1.1
    maxy1m = max(df4["percentile75"])*1.1
    maxy6m = max(df5["percentile75"])*1.1

    fig = go.Figure()

    for ix, i in enumerate(df["relay"].drop_duplicates()):
        _df = df[df["relay"] == i]
        #yaxismax = [0, maxy]
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            customdata = _df["c"],
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ))
        )
        _df = df2[df2["relay"] == i]
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )
        )
        _df = df3[df3["relay"] == i]
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )              
        )
        _df = df4[df4["relay"] == i]
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )              
        )
        _df = df5[df5["relay"] == i]
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )              
        )
    h24 = [True if i%5==0 else False for i in range(len(fig.data))]
    h48 = [True if i%5==1 else False for i in range(len(fig.data))]
    hw1 = [True if i%5==2 else False for i in range(len(fig.data))]
    hm1 = [True if i%5==3 else False for i in range(len(fig.data))]
    hm6 = [True if i%5==4 else False for i in range(len(fig.data))]
    updatemenus=[
            dict(
                bgcolor = "#ffffff",
                #type="buttons",
                pad={"r": 0, "t": 0},
                showactive=True,
                x=1,
                xanchor="right",
                y=1.2,
                yanchor="top",
                buttons=list(
                    [dict(label='24 hours',
                         method="update",
                         args=[{"visible": h24},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 24 hours)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy24],
                                "annotations": []}])]+
                    [dict(label='48 hours',
                         method="update",
                         args=[{"visible": h48},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 48 hours)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy48],
                                "annotations": []}])]+
                    [dict(label='7 days',
                         method="update",
                         args=[{"visible": hw1},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 7 days)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy1w],
                                "annotations": []}])]+
                    [dict(label='30 days',
                         method="update",
                         args=[{"visible": hm1},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 30 days)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy1m],
                                "annotations": []}])]
                    +
                    [dict(label='6 months',
                         method="update",
                         args=[{"visible": hm6},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 6 months)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy6m],
                                "annotations": []}])]
                ))]
    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 24 hours)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title='ETH',
        margin={"l":100, "t":70},
        hovermode="closest",
        yaxis_tickformat = "0.1r",
        yaxis=dict(
            #autorange=True,
            showgrid=True,
            zeroline=True,
            gridcolor =  BLACK_ALPHA.format(0.2),
            gridwidth=0.2,
            zerolinecolor='rgb(255, 255, 255)',
            zerolinewidth=2,
            #type="log",
            range = [0, maxy24],
            #fixedrange= True
        ),
        xaxis=dict(
            fixedrange= True
        ),
        plot_bgcolor='#eee',
        showlegend=False,
        font=dict(
            family="Courier New, monospace",
            size=18,
            color=BLACK
        ),
        #autosize=True, 
        height=500,
        updatemenus=updatemenus
    )
    fig.update_yaxes(tickfont=dict(size=16),title_font=dict(size=20))
    fig.update_xaxes(tickfont=dict(size=16),title_font=dict(size=20))
    for ix, i in enumerate(fig.data):            
        i["hoverlabel"]["bgcolor"] = i["fillcolor"].replace(",0.8", ",0.2")
    fig.update_traces(hoverlabel=dict(font=dict(color=BLACK, size=16)))
    
    return fig
    
create_avg_proposer_payment_relay_boxes_chart = create_avg_proposer_payment_relay_boxes()
if JUPYTER:
    create_avg_proposer_payment_relay_boxes_chart.show()

In [ ]:
def create_avg_proposer_payment_builder_boxes():  
    print("start create_avg_proposer_payment_builder_boxes")
    df = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_builder_boxplot_24h`')
    df["builder"] = df["builder"].apply(lambda x: str(x[0].upper()+x[1:13]).replace("Https://", ""))
    df2 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_builder_boxplot_48h`')
    df2["builder"] = df2["builder"].apply(lambda x: str(x[0].upper()+x[1:13]).replace("Https://", ""))
    df3 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_builder_boxplot_1w`')
    df3["builder"] = df3["builder"].apply(lambda x: str(x[0].upper()+x[1:13]).replace("Https://", ""))
    df4 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_builder_boxplot_1m`')
    df4["builder"] = df4["builder"].apply(lambda x: str(x[0].upper()+x[1:13]).replace("Https://", ""))
    df5 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_builder_boxplot_6m`')
    df5["builder"] = df5["builder"].apply(lambda x: str(x[0].upper()+x[1:13]).replace("Https://", ""))

    
    maxy24 = max(df["percentile75"])*1.1
    maxy48 = max(df2["percentile75"])*1.1
    maxy1w = max(df3["percentile75"])*1.1
    maxy1m = max(df4["percentile75"])*1.1
    maxy6m = max(df5["percentile75"])*1.1

    fig = go.Figure()

    for ix, i in enumerate(pd.concat([df, df3]).groupby("builder")["c"].sum().reset_index().sort_values("c", ascending=False)["builder"].drop_duplicates()):
        _df = df[df["builder"] == i]
        if len(_df) == 0:
            fig.add_trace(go.Box())
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            customdata = _df["c"],
            line_width=1,
            name = i.split("(")[0] if "(" in i else i,
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ))
        )
    
        _df = df2[df2["builder"] == i]
        if len(_df) == 0:
            fig.add_trace(go.Box())
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.split("(")[0] if "(" in i else i,
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )
        )
        _df = df3[df3["builder"] == i]
        if len(_df) == 0:
            fig.add_trace(go.Box())
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name =i.split("(")[0] if "(" in i else i,
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )
        )
        _df = df4[df4["builder"] == i]
        if len(_df) == 0:
            fig.add_trace(go.Box())
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )              
        )
        _df = df5[df5["builder"] == i]
        if len(_df) == 0:
            fig.add_trace(go.Box())
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )              
        )

            
                       
    h24 = [True if i%5==0 else False for i in range(len(fig.data))]
    h48 = [True if i%5==1 else False for i in range(len(fig.data))]
    hw1 = [True if i%5==2 else False for i in range(len(fig.data))]
    hm1 = [True if i%5==3 else False for i in range(len(fig.data))]
    hm6 = [True if i%5==4 else False for i in range(len(fig.data))]
    updatemenus=[
            dict(
                bgcolor = "#ffffff",
                #type="buttons",
                pad={"r": 0, "t": 0},
                showactive=True,
                x=1,
                xanchor="right",
                y=1.2,
                yanchor="top",
                buttons=list(
                    [dict(label='24 hours',
                         method="update",
                         args=[{"visible": h24},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 24 hours)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy24],
                                "annotations": []}])]+
                    [dict(label='48 hours',
                         method="update",
                         args=[{"visible": h48},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 48 hours)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy48],
                                "annotations": []}])]+
                    [dict(label='7 days',
                         method="update",
                         args=[{"visible": hw1},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 7 days)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy1w],
                                "annotations": []}])]+
                    [dict(label='30 days',
                         method="update",
                         args=[{"visible": hm1},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 30 days)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy1m],
                                "annotations": []}])]+
                    [dict(label='6 months',
                         method="update",
                         args=[{"visible": hm6},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 6 months)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy6m],
                                "annotations": []}])]
                ))]
    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 24 hours)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title='ETH',
        margin={"l":100, "t":70},
        hovermode="closest",
        yaxis_tickformat = "0.1r",
        yaxis=dict(
            #autorange=True,
            showgrid=True,
            zeroline=True,
            gridcolor =  BLACK_ALPHA.format(0.2),
            gridwidth=0.2,
            zerolinecolor='rgb(255, 255, 255)',
            zerolinewidth=2,
            #type="log",
            range = [0, maxy24],
            #fixedrange= True
        ),
        xaxis=dict(
            fixedrange= True
        ),
        plot_bgcolor='#eee',
        showlegend=False,
        font=dict(
            family="Courier New, monospace",
            size=18,
            color=BLACK
        ),
        #autosize=True, 
        height=500,
        updatemenus=updatemenus
    )
    fig.update_yaxes(tickfont=dict(size=16),title_font=dict(size=20))
    fig.update_xaxes(tickfont=dict(size=16),title_font=dict(size=20))
    for ix, i in enumerate(fig.data): 
        try:
            i["hoverlabel"]["bgcolor"] = i["fillcolor"].replace(",0.8", ",0.2")
        except:
            pass
    fig.update_traces(hoverlabel=dict(font=dict(color=BLACK, size=16)))
    
    return fig
    
create_avg_proposer_payment_builder_boxes_chart = create_avg_proposer_payment_builder_boxes()
if JUPYTER:
    create_avg_proposer_payment_builder_boxes_chart.show()

In [ ]:
def create_avg_proposer_payment_validator_boxes():  
    print("start create_avg_proposer_payment_validator_boxes")
    df = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_validator_boxplot_24h`').replace("ultrasound", "ultra Sound")
    df["validator"] = df["validator"].apply(lambda x: str(x[0].upper()+x[1:14]).replace("https://", "")) 
    df2 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_validator_boxplot_48h`').replace("ultrasound", "ultra Sound")
    df2["validator"] = df2["validator"].apply(lambda x: str(x[0].upper()+x[1:14]).replace("https://", "")) 
    df3 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_validator_boxplot_1w`').replace("ultrasound", "ultra Sound")
    df3["validator"] = df3["validator"].apply(lambda x: str(x[0].upper()+x[1:14]).replace("https://", "")) 
    df4 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_validator_boxplot_1m`').replace("ultrasound", "ultra Sound")
    df4["validator"] = df4["validator"].apply(lambda x: str(x[0].upper()+x[1:14]).replace("https://", "")) 
    df5 = pd.read_gbq('SELECT * FROM `ethereum-data-nero.eth.6_validator_boxplot_6m`').replace("ultrasound", "ultra Sound")
    df5["validator"] = df5["validator"].apply(lambda x: str(x[0].upper()+x[1:14]).replace("https://", "")) 
    
    maxy24 = max(df["percentile75"])*1.1
    maxy48 = max(df2["percentile75"])*1.1
    maxy1w = max(df3["percentile75"])*1.1
    maxy1m = max(df4["percentile75"])*1.1
    maxy6m = max(df5["percentile75"])*1.1

    fig = go.Figure()

    for ix, i in enumerate(pd.concat([df, df3]).groupby("validator")["c"].sum().reset_index().sort_values("c", ascending=False)["validator"].drop_duplicates()):
        _df = df[df["validator"] == i]
        if len(_df) == 0:
            continue
        #yaxismax = [0, maxy]
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            customdata = _df["c"],
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ))
        )
    
        _df = df2[df2["validator"] == i]
        if len(_df) == 0:
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )
        )

        _df = df3[df3["validator"] == i]
        if len(_df) == 0:
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )
        )
        _df = df4[df4["validator"] == i]
        if len(_df) == 0:
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )              
        )
        _df = df5[df5["validator"] == i]
        if len(_df) == 0:
            continue
        fig.add_trace(go.Box(
            y=[float(_df["min"]), 
            float(_df["percentile25"]), 
            float(_df["percentile25"]), 
            float(_df["median"]), 
            float(_df["percentile75"]), 
            float(_df["percentile75"]), 
            float(_df["max"])],
            quartilemethod="linear",
            boxpoints=False,
            line_width=1,
            name = i.replace("(","<br>("),
            fillcolor=[hextorgba(j[1:], 0.8) for j in colors*2][ix],
            whiskerwidth=1,
            line=dict(
                color=BLACK,  # Change this to the desired border color
                width=1
            ),
        visible=False
        )              
        )

            
    h24 = [True if i%5==0 else False for i in range(len(fig.data))]
    h48 = [True if i%5==1 else False for i in range(len(fig.data))]
    hw1 = [True if i%5==2 else False for i in range(len(fig.data))]
    hm1 = [True if i%5==3 else False for i in range(len(fig.data))]
    hm6 = [True if i%5==4 else False for i in range(len(fig.data))]
    updatemenus=[
            dict(
                bgcolor = "#ffffff",
                #type="buttons",
                pad={"r": 0, "t": 0},
                showactive=True,
                x=1,
                xanchor="right",
                y=1.2,
                yanchor="top",
                buttons=list(
                    [dict(label='24 hours',
                         method="update",
                         args=[{"visible": h24},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 24 hours)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy24],
                                "annotations": []}])]+
                    [dict(label='48 hours',
                         method="update",
                         args=[{"visible": h48},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 48 hours)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy48],
                                "annotations": []}])]+
                    [dict(label='7 days',
                         method="update",
                         args=[{"visible": hw1},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 7 days)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy1w],
                                "annotations": []}])]+
                    [dict(label='30 days',
                         method="update",
                         args=[{"visible": hm1},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 30 days)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy1m],
                                "annotations": []}])]+
                    [dict(label='6 months',
                         method="update",
                         args=[{"visible": hm6},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 6 months)</span>',
                                 "height":500,
                                "yaxis.range": [0, maxy6m],
                                "annotations": []}])]
                ))]
    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 24 hours)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title='ETH',
        margin={"l":100, "t":70},
        hovermode="closest",
        yaxis_tickformat = "0.1r",
        yaxis=dict(
            #autorange=True,
            showgrid=True,
            zeroline=True,
            gridcolor =  BLACK_ALPHA.format(0.2),
            gridwidth=0.2,
            zerolinecolor='rgb(255, 255, 255)',
            zerolinewidth=2,
            #type="log",
            range = [0, maxy24],
            #fixedrange= True
        ),
        xaxis=dict(
            fixedrange= True
        ),
        plot_bgcolor='#eee',
        showlegend=False,
        font=dict(
            family="Courier New, monospace",
            size=18,
            color=BLACK
        ),
        #autosize=True, 
        height=500,
        updatemenus=updatemenus
    )
    fig.update_yaxes(tickfont=dict(size=16),title_font=dict(size=20))
    fig.update_xaxes(tickfont=dict(size=16),title_font=dict(size=20))
    for ix, i in enumerate(fig.data):            
        i["hoverlabel"]["bgcolor"] = i["fillcolor"].replace(",0.8", ",0.2")
    fig.update_traces(hoverlabel=dict(font=dict(color=BLACK, size=16)))
    
    return fig
    
create_avg_proposer_payment_validator_boxes_chart = create_avg_proposer_payment_validator_boxes()
if JUPYTER:
    create_avg_proposer_payment_validator_boxes_chart.show()

In [ ]:
def create_bids_relay_share_chart():
    print("start create_bids_relay_share_chart")
    df = pd.read_gbq('SELECT distinct  * FROM ethereum-data-nero.eth.8_bids_relay_over_time order by submissions desc')
    df["relay"] = df["relay"].apply(lambda x: x[0].upper()+x[1:]) 

    df = df.replace("Ultrasound", "Ultra Sound")
    if df[df["date"] == max(df["date"])]["submissions"].sum() < 0.7 * df[df["date"] == sorted(list(set(df["date"])))[-2]]["submissions"].sum():
        df = df[df["date"] != max(df["date"])]
        
    df["relay"] = df["relay"].apply(lambda x: x.replace("(", "<br>("))
    

    patternmap = [(i, "+") for i in df["relay"].drop_duplicates()]
    fig = px.bar(df[["date", "submissions", "relay"]], x="date", 
                  y="submissions", 
                  color="relay", 
                  color_discrete_sequence=[hextorgba(j[1:], 0.7) for j in colors],
                 pattern_shape="relay", pattern_shape_map=dict(patternmap),
                text="relay")

    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Bids received (last 48 hours)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="bids received",
        legend_title="Relay Provider",

        #title_xanchor="left",
        #title_yanchor="auto",

        margin={"l":100, "t":70},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        paper_bgcolor='#eee',
        plot_bgcolor="rgba(255,255,255,0.9)",
        #yaxis=dict(fixedrange =True),
        autosize=True, 
        #height=580,
        #width=1600,
        height=590

    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(hovertemplate="<b>%{text}: %{y:,.0f} bids<br>%{x}<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16)),textposition = "none")
    #fig.update_traces(width=1000)
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))

    fig.update_layout(bargap=0.1,
                      bargroupgap = 0.01,
                     )
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    fig.update_xaxes(tickfont=dict(size=18), title_font=dict(size=20))
    fig.update_yaxes(tickfont=dict(size=18), title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color(px.colors.qualitative.Plotly[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color(px.colors.qualitative.Plotly[ix],0.2)
       


    return fig
create_bids_relay_share_chart_chart = create_bids_relay_share_chart()

if JUPYTER:
    create_bids_relay_share_chart_chart.show()

In [ ]:
def create_bids_builder_share_chart():
    print("start create_bids_builder_share_chart")
    df = pd.read_gbq('SELECT distinct  * FROM ethereum-data-nero.eth.8_bids_builder_over_time')

    def give_label(x):
        if x == None:
            return ""
        return "<br>(" + x[:15]+")"
    order = df.groupby("label", as_index=False)["submissions"].sum().sort_values("submissions", ascending=False)["label"].values.tolist()
    df = df[df["label"].isin(order[:15])]

    df = df.set_index("label")
    df = df.loc[order[:15]]
    df = df.reset_index()
    df["builder_pubkey"] = df.apply(lambda x: x["builder_pubkey"][0:9] +"... " +give_label(x["label"]), axis=1)


    if df[df["date"] == max(df["date"])]["submissions"].sum() < 0.7 * df[df["date"] == sorted(list(set(df["date"])))[-2]]["submissions"].sum():
        df = df[df["date"] != max(df["date"])]

    patternmap = [(i, "+") for i in df["builder_pubkey"].drop_duplicates()]
    fig = px.bar(df[["date", "submissions", "builder_pubkey"]], x="date", 
                  y="submissions", 
                  color="builder_pubkey", 
                  color_discrete_sequence=[hextorgba(j[1:], 0.8) for j in colors],
                 pattern_shape="builder_pubkey", pattern_shape_map=dict(patternmap),
                text="builder_pubkey")

    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Bids submitted (last 24 hours)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="nr of bids",
        legend_title="Block Builder",

        #title_xanchor="left",
        #title_yanchor="auto",

        margin={"l":100, "t":70},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        #paper_bgcolor='#eee',
        plot_bgcolor="rgba(255,255,255,0.9)",
        #yaxis=dict(fixedrange =True),
        autosize=True, 
        #height=580,
        #width=1600,
        height=580

    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(hovertemplate="<b>%{text}: %{y:,.0f} bids<br>%{x}<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16)),textposition = "none")
    #fig.update_traces(width=1000)
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))

    fig.update_layout(bargap=0.1,
                      bargroupgap = 0.01,
                     )
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    fig.update_xaxes(tickfont=dict(size=18), title_font=dict(size=20))
    fig.update_yaxes(tickfont=dict(size=18), title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*20)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*20)[ix],0.2)
        if i.legendgroup == "Other/None":
            #i.fillpattern["size"]= 5
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
            continue


    def get_visibility(forentity):
        visiblelist=[]
        bnkeys = df[df["label"] == forentity]["builder_pubkey"].values.tolist()
        for i in df["builder_pubkey"].drop_duplicates():
            visiblelist.extend([True if i in bnkeys else False])
        return visiblelist

    visibilities = {}
    for i in df["label"].drop_duplicates():
        visibilities[i] = get_visibility(i)

    updatemenus=[
            dict(
                bgcolor = "#ffffff",
                #type="buttons",
                pad={"r": -210, "t": 0},
                showactive=True,
                x=1,
                xanchor="right",
                y=1.2,
                yanchor="top",
                buttons=list(
                    [dict(label='All Builders',
                         method="update",
                         args=[{"visible": [True]},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">Total Bids (MEV-Boost)</span>',
                                 "height":580,
                                "annotations": []}])] +
                    [dict(label=f"{j}",
                         method="update",
                         args=[{"visible": visibilities[j]},
                               {f"title": f'<span style="font-size: 24px;font-weight:bold;">Bids of `{j}`</span>',
                                 "height":580,
                                "annotations": []}]) for j in visibilities]
                ))]


    fig.update_layout(font_size=16,
                      paper_bgcolor='#eee',
                      font=dict(
                                family="Courier New, monospace",
                                #size=24,  # Set the font size here
                                color=BLACK
                               ), autosize=False, width=800, height=580,
                      updatemenus=updatemenus)
    return fig

create_bids_builder_share_chart_chart = create_bids_builder_share_chart()

if JUPYTER:
    create_bids_builder_share_chart_chart.show()

In [ ]:
DS = "ethereum-data-nero.eth.3_relays_over_time"
query = build_query("timestamp, relay, slot", DS, "ORDER BY slot DESC")
df_relays = pd.read_gbq(query)

nrofrelayers = len(set(df_relays["relay"])) - 1
len(df_relays[df_relays["relay"]!="Other/None"])
len(df_relays[df_relays["relay"]=="Other/None"])
LAST_SLOT = get_last_known_slot()
LAST_TS = get_timestamp_of_slot(LAST_SLOT)

def create_mev_share_chart():
    print("start create_mev_share_chart")
    df = df_relays
    df = df.replace("Other/None", "Vanilla Builders")
    df_copy = pd.DataFrame(df[df["relay"] == "Vanilla Builders"].groupby("timestamp")["slot"].sum()).reset_index()
    df_copy["relay"] = "Vanilla Builders"
    df_copy2 = pd.DataFrame(df[df["relay"] != "Vanilla Builders"].groupby("timestamp")["slot"].sum()).reset_index()
    df_copy2["relay"] = "MEV-Boost"
    df = pd.concat([df_copy, df_copy2], ignore_index=True)
    relays = [i for i in df["relay"].drop_duplicates()]
    relays = relays[1:]+[relays[0]]
    df = df.set_index("relay")
    df = df.loc[relays]
    df = df.reset_index()
    df = df[df["timestamp"] != max(df["timestamp"])]
    time_axis = df["timestamp"].drop_duplicates().values.tolist()[::int(len(df["timestamp"].drop_duplicates())/10)][:-1]+[df["timestamp"].drop_duplicates().iloc[-1]]
    patternmap = [(i, "+") for i in df["relay"].drop_duplicates()]
    #patternmap.remove(('Vanilla Builders', None))
    #patternmap.append(('Vanilla Builders', "+"))
    fig = px.area(df, x="timestamp", 
                  y="slot", 
                  color="relay", 
                  line_group="relay",
                  groupnorm="percent",
                  color_discrete_map={'Vanilla Builders': "rgb(82, 170, 231)", 'MEV-Boost': "rgb(255, 153, 62)"},
                  pattern_shape="relay", pattern_shape_map=dict(patternmap)
                 )
    fig.update_layout(
        xaxis_tickangle=-45,
        title='<span style="font-size: 24px;font-weight:bold;">MEV-Boost Slot Share</span>',
        xaxis_title=None,
        yaxis_title=None, #"% of total slots",
        legend_title="",
        legend=dict(
                bgcolor="rgba(255,255,255,0.9)",
                yanchor="bottom",
                y=0.01,
                xanchor="right",
                x=0.99,
                font=dict(
                    family="Courier New, monospace",
                    size=18,  
                    color=BLACK
                ),
            ),
        hovermode="x unified",
        #title_xanchor="left",
        #title_yanchor="auto",
        
        margin={"l":100, "t":100},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        paper_bgcolor="#eee",
        plot_bgcolor="rgba(255,255,255,0.1)",
        hoverlabel=dict(
            font_size=16,
            font_family="Courier New, monospace"
        ),
        #yaxis=dict(fixedrange =True),
        autosize=True, 
        height=500,
        yaxis=dict(
            tickvals = [20, 40, 60, 80, 100],
            ticktext= ["20%", "40%", "60%", "80%", "100%"],
            range = [0,100]
        ),
        xaxis=dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = BLACK,
                rangeselector=dict(
                    buttons=list([
                        dict(count=7,
                             label="7d",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        #dict(count=6,
                        #     label="6m",
                        #     step="month",
                        #     stepmode="backward"),
                        dict(step="all", label="since merge")
                    ])
                ),
                
            type="date"
        )
        
    )
    fig.update_traces(hovertemplate="<b>%{y:,.0f}%<b>", mode='lines', line=dict(width=0.8))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
                    #title_font=dict(size=16))
    for i in fig["data"]:
        if i.legendgroup == "Vanilla Builders":
            i.fillpattern["size"]= 5
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
    return fig
create_mev_share_chart_chart = create_mev_share_chart()

if JUPYTER:
    create_mev_share_chart_chart.show()

In [ ]:
def create_total_share_chart():
    print("start create_total_share_chart")
    DS = "ethereum-data-nero.eth.3_relays_over_time"
    query = build_query("timestamp, relay, slot", DS, "ORDER BY slot DESC")
    df = pd.read_gbq(query)
    relays = [i for i in df["relay"].drop_duplicates()]
    #    relays = [i for i in df[df["timestamp"] == max(df["timestamp"])]["relay"].drop_duplicates()]
    relays = relays[1:]+[relays[0]]
    df = df.set_index("relay")
    df = df.loc[relays]
    df = df.reset_index()
    df["relay"] = df["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    df = df.replace("Ultrasound", "Ultra Sound")
    df = df[df["timestamp"] != max(df["timestamp"])]
    fdf = pd.DataFrame()
    ddf = pd.DataFrame(df[df["relay"]=="Other/None"]["timestamp"])
    for i in df["relay"].drop_duplicates():
        _df = df[df["relay"] == i]
        _df = pd.merge(ddf, _df, on="timestamp", how="left").fillna(0)
        _df.relay = i
        fdf = pd.concat([fdf, _df], ignore_index=True)
    time_axis = df["timestamp"].drop_duplicates().values.tolist()[::int(len(df["timestamp"].drop_duplicates())/10)][:-1]+[df["timestamp"].drop_duplicates().iloc[-1]]
    patternmap = [(i, None) for i in df["relay"].drop_duplicates()]
    patternmap.remove(('Other/None', None))
    patternmap.append(('Other/None', "+"))
    fig = px.area(fdf, x="timestamp", 
                  y="slot", 
                  color="relay", 
                  line_group="relay",
                  groupnorm="percent",
                  color_discrete_sequence=px.colors.qualitative.Plotly,
                  pattern_shape="relay", pattern_shape_map=dict(patternmap),
                  text="relay"
                 )
    fig.update_layout(
        xaxis_tickangle=-45,
        title='<span style="font-size: 24px;font-weight:bold;">Total Slot Share (cumulative)</span>',
        xaxis_title="",
        yaxis_title="% of total slots",
        #yaxis_range = [0,100],
        legend_title="Relay Provider",
       
        #title_xanchor="left",
        #title_yanchor="auto",
        
        margin={"l":100, "t":100},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        paper_bgcolor='#eee',
        plot_bgcolor='#ffffff',
        #yaxis=dict(fixedrange =True),
        autosize=True, 
        height=580,
        xaxis=dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = BLACK,
                rangeselector=dict(
                    buttons=list([
                        dict(count=7,
                             label="7d",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        #dict(count=6,
                        #     label="6m",
                        #     step="month",
                        #     stepmode="backward"),
                        dict(step="all", label="since merge")
                    ])
                ),
                
            type="date"
        )
        
    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(mode='lines', line=dict(width=0.8))
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                    title_font=dict(size=20), range=[0,100])
    
    fig.update_traces(hoverinfo="skip",hovertemplate="<b>%{x}<br>%{text}: %{y:.0f} %<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16))
                     )
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color(px.colors.qualitative.Plotly[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color(px.colors.qualitative.Plotly[ix],0.2)
            
    for i in fig["data"]:
        if i.legendgroup == "Other/None":
            i.fillpattern["size"]= 5
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
    return fig
create_total_share_chart_chart = create_total_share_chart()
if JUPYTER:
    create_total_share_chart_chart.show()

In [ ]:
def create_total_relay_share_per_hour_chart():
    print("start create_total_relay_share_per_hour_chart")
    _colors = colors[0:colors.index('#02e382')]+colors[colors.index('#02e382')+1:]
    DS = "ethereum-data-nero.eth.3_relays_over_time_last_7_days"
    query = build_query("timestamp, relay, slot", DS, "ORDER BY slot desc")
    df_relay = pd.read_gbq(query)
    df_relay = pd.merge(df_relay, df_relay[["timestamp", "slot"]].groupby("timestamp").sum().reset_index(), on="timestamp", how="right")
    df_relay["per"] = df_relay["slot_x"] / df_relay["slot_y"]*100
    df_relay = df_relay.rename(columns={"slot_x": "slot"})
    df_relay = df_relay[~df_relay["timestamp"].isin(df_relay.sort_values("timestamp")["timestamp"].drop_duplicates().tail(2))]
    df_relay["relay"] = df_relay["relay"].apply(lambda x: x[0:12] if x.startswith("0x") else x)
    df_relay["relay"] = df_relay["relay"].apply(lambda x: x.replace("(", "<br>("))
    df_relay = df_relay.replace("ultrasound", "Ultra Sound")
    _df = df_relay.sort_values("slot", ascending=False)["relay"].drop_duplicates().tolist()
    _df.remove("Other/None")
    df_relay = df_relay.set_index('relay')
    df_relay = df_relay.loc[_df+ ["Other/None"]].reset_index()
    df_relay["relay"] = df_relay["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    max_entry = df_relay[df_relay["timestamp"] == max(df_relay["timestamp"])]
    if sum(max_entry[max_entry["relay"] == "Other/None"]["per"]) / 2 > sum(max_entry[max_entry["relay"] != "Other/None"]["per"]):
        df_relay = df_relay[df_relay["timestamp"] != max(df_relay["timestamp"])]

    patternmap = [(i, None) for i in df_relay["relay"].drop_duplicates()]
    patternmap.remove(('Other/None', None))
    patternmap.append(('Other/None', "+"))
    fig = px.bar(df_relay[["timestamp", "per", "relay"]], x="timestamp", 
                  y="per", 
                  color="relay", 
                  color_discrete_sequence=[hextorgba(j[1:], 0.8) for j in _colors],
                 pattern_shape="relay", pattern_shape_map=dict(patternmap),
                text="relay")
   
    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Slots per Hour (last 7 days)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="% of total slots",
        legend_title="Relay Provider",
       
        #title_xanchor="left",
        #title_yanchor="auto",
        
        margin={"l":100, "t":70},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        #paper_bgcolor='#eee',
        plot_bgcolor="rgba(255,255,255,0.9)",
        #yaxis=dict(fixedrange =True),
        autosize=True, 
        #height=580,
        #width=1600,
        height=590
        
    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(hovertemplate="<b>%{text}: %{y:.0f}%<br>%{x}<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16)),textposition = "none")
    #fig.update_traces(width=1000)
    #fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    
    fig.update_layout(bargap=0.1,
                      bargroupgap = 0.01,
                     )
    #fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2), range=[0,100])
    fig.update_xaxes(tickfont=dict(size=18), title_font=dict(size=20))
    fig.update_yaxes(tickfont=dict(size=18), title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color(_colors[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color(_colors[ix],0.2)
        if i.legendgroup == "Other/None":
            #i.fillpattern["size"]= 5
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
            continue
    return fig
create_total_relay_share_per_hour_chart_chart = create_total_relay_share_per_hour_chart()
if JUPYTER:
    create_total_relay_share_per_hour_chart_chart.show()

In [ ]:
def create_validator_share_chart():
    print("start create_validator_share_chart")
    DS = "ethereum-data-nero.eth.3_validators_over_time"
    query = build_query("timestamp, validator, slot", DS, "ORDER BY slot DESC")
    df = pd.read_gbq(query)
    validators = [i for i in df["validator"].drop_duplicates()]
    #validators = validators[1:]+[validators[0]]
    #df = df.set_index("validator")
    #df = df.loc[validators]
    #df = df.reset_index()
    df["validator"] = df["validator"].apply(lambda x: x[0].upper()+x[1:]) 
    fdf = pd.DataFrame()
    ddf = pd.DataFrame(df[df["validator"]=="Others"]["timestamp"])
    for i in df["validator"].drop_duplicates():
        _df = df[df["validator"] == i]
        _df = pd.merge(ddf, _df, on="timestamp", how="left").fillna(0)
        _df.validator = i
        fdf = pd.concat([fdf, _df], ignore_index=True)
    time_axis = df["timestamp"].drop_duplicates().values.tolist()[::int(len(df["timestamp"].drop_duplicates())/10)][:-1]+[df["timestamp"].drop_duplicates().iloc[-1]]
    patternmap = [(i, None) for i in df["validator"].drop_duplicates()]
    #patternmap.remove(('Other/None', None))
    #patternmap.append(('Other/None', "+"))
    fig = px.area(fdf, x="timestamp", 
                  y="slot", 
                  color="validator", 
                  line_group="validator",
                  groupnorm="percent",
                  color_discrete_sequence=[hextorgba(j[1:], 1) for j in colors],
                  pattern_shape="validator", pattern_shape_map=dict(patternmap),
                  text="validator"
                 )
    fig.update_layout(
        xaxis_tickangle=-45,
        title='<span style="font-size: 24px;font-weight:bold;">Total Slot Share (cumulative)</span>',
        xaxis_title="",
        yaxis_title="% of total slots",
        legend_title="Validators",
       
        #title_xanchor="left",
        #title_yanchor="auto",
        
        margin={"l":100, "t":100},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        paper_bgcolor='#eee',
        #yaxis=dict(fixedrange =True),
        autosize=True, 
        height=580,
        xaxis=dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = BLACK,
                rangeselector=dict(
                    buttons=list([
                        dict(count=7,
                             label="7d",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        #dict(count=6,
                        #     label="6m",
                        #     step="month",
                        #     stepmode="backward"),
                        dict(step="all", label="since merge")
                    ])
                ),
                
            type="date"
        )
        
    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(mode='lines', line=dict(width=0.8))
    fig.update_traces(hoverinfo="skip",hovertemplate="<b>%{x}<br>%{text}: %{y:.0f} %<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16)))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                    title_font=dict(size=20), range=[0,100])
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*10)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*10)[ix],0.2)
            
        if i.legendgroup == "Other/None":
            i.fillpattern["size"]= 5
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
    return fig
create_validator_share_chart_chart = create_validator_share_chart()
if JUPYTER:
    create_validator_share_chart_chart.show()

In [ ]:
def create_lido_validator_share_chart():
    print("start create_lido_validator_share_chart")
    DS = "ethereum-data-nero.eth.3_lido_validators_over_time"
    query = build_query("timestamp, validator, slot", DS, "ORDER BY slot DESC")
    df = pd.read_gbq(query)
    validators = [i for i in df["validator"].drop_duplicates()]
    #validators = validators[1:]+[validators[0]]
    #df = df.set_index("validator")
    #df = df.loc[validators]
    #df = df.reset_index()
    df["validator"] = df["validator"].apply(lambda x: x[0].upper()+x[1:]) 
    time_axis = df["timestamp"].drop_duplicates().values.tolist()[::int(len(df["timestamp"].drop_duplicates())/10)][:-1]+[df["timestamp"].drop_duplicates().iloc[-1]]
    patternmap = [(i, None) for i in df["validator"].drop_duplicates()]
    #patternmap.remove(('Other/None', None))
    #patternmap.append(('Other/None', "+"))
    fig = px.area(df, x="timestamp", 
                  y="slot", 
                  color="validator", 
                  line_group="validator",
                  groupnorm="percent",
                  color_discrete_sequence=[hextorgba(j[1:], 0.8) for j in colors],
                  pattern_shape="validator", pattern_shape_map=dict(patternmap),
                   hover_data={'timestamp':False},
                  text="validator"
                 )
    fig.update_layout(
        xaxis_tickangle=-45,
        title='<span style="font-size: 24px;font-weight:bold;">Lido Node Operator Slot Share (cumulative)</span>',
        xaxis_title="",
        yaxis_title="% of total slots",
        legend_title="Node Operators",       
        #title_xanchor="left",
        #title_yanchor="auto",
        
        margin={"l":100, "t":100},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        paper_bgcolor='#eee',
        plot_bgcolor='#ffffff',
        #yaxis=dict(fixedrange =True),
        autosize=True, 
        height=580,
        xaxis=dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = BLACK,
                rangeselector=dict(
                    buttons=list([
                        dict(count=7,
                             label="7d",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        #dict(count=6,
                        #     label="6m",
                        #     step="month",
                        #     stepmode="backward"),
                        dict(step="all", label="since merge")
                    ])
                ),
                
            type="date"
        )
        
    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(hovertemplate="<b>%{x}<br>%{text}: %{y:.0f} %<extra></extra><b>", mode='lines', line=dict(width=0.8),
                      hoverlabel=dict(font=dict(color=BLACK, size=16)))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                    title_font=dict(size=20), range=[0,100])
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.2)
        if i.legendgroup == "Other/None":
            i.fillpattern["size"]= 5
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
    return fig
create_lido_validator_share_chart_chart = create_lido_validator_share_chart()
if JUPYTER:
    create_lido_validator_share_chart_chart.show()

In [ ]:
def create_total_validator_share_per_hour_chart():
    print("start create_total_validator_share_per_hour_chart")
    DS = "ethereum-data-nero.eth.3_validators_over_time_last_7_days"
    query = build_query("timestamp, validator, slot", DS, "ORDER BY slot desc")
    df_validator = pd.read_gbq(query)
    df_validator = pd.merge(df_validator, df_validator[["timestamp", "slot"]].groupby("timestamp").sum().reset_index(), on="timestamp", how="right")
    df_validator["per"] = df_validator["slot_x"] / df_validator["slot_y"]*100
    df_validator = df_validator.rename(columns={"slot_x": "slot"})
    df_validator = df_validator[~df_validator["timestamp"].isin(df_validator.sort_values("timestamp")["timestamp"].drop_duplicates().tail(2))]
    df_validator["validator"] = df_validator["validator"].apply(lambda x: x[0:12] if x.startswith("0x") else x[0:17])
    df_validator["validator"] = df_validator["validator"].apply(lambda x: x[0].upper()+x[1:]) 
    
    fig = px.bar(df_validator[["timestamp", "per", "validator"]], x="timestamp", 
                  y="per", 
                  color="validator", 
                  color_discrete_sequence=[hextorgba(j[1:], 0.8) for j in colors],
                  text="validator"
                 )
   
    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Slots per Hour (last 7 days)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="% of total slots",
        legend_title="Validators",
       
        #title_xanchor="left",
        #title_yanchor="auto",
        
        margin={"l":100, "t":70},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        #paper_bgcolor='#eee',
        plot_bgcolor="rgba(255,255,255,0.9)",

        #yaxis=dict(fixedrange =True),
        autosize=True, 
        #height=580,
        #width=1600,
        height=580,
       
        
    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(hovertemplate="<b>%{text}: %{y:.0f} %<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16)),textposition = "none")
    #fig.update_traces(width=1000)
    #fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    
    fig.update_layout(bargap=0.1,
                      bargroupgap = 0.01,
                     )
    fig.update_yaxes(title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*10)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*10)[ix],0.2)
        if i.legendgroup == "Other/None":
            #i.fillpattern["size"]= 5
            pass
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
    return fig
create_total_validator_share_per_hour_chart_chart = create_total_validator_share_per_hour_chart()
if JUPYTER:
    create_total_validator_share_per_hour_chart_chart.show()

In [ ]:
def create_total_lido_validator_share_per_hour_chart():
    print("start create_total_lido_validator_share_per_hour_chart")
    DS = "ethereum-data-nero.eth.3_lido_validators_over_time_last_7_days"
    query = build_query("timestamp, validator, slot", DS, "ORDER BY slot desc")
    df_validator = pd.read_gbq(query)
    df_validator = pd.merge(df_validator, df_validator[["timestamp", "slot"]].groupby("timestamp").sum().reset_index(), on="timestamp", how="right")
    df_validator["per"] = df_validator["slot_x"] / df_validator["slot_y"]*100
    df_validator = df_validator.rename(columns={"slot_x": "slot"})
    df_validator = df_validator[~df_validator["timestamp"].isin(df_validator.sort_values("timestamp")["timestamp"].drop_duplicates().tail(2))]
    df_validator["validator"] = df_validator["validator"].apply(lambda x: x[0:12] if x.startswith("0x") else x[0:17])
    df_validator["validator"] = df_validator["validator"].apply(lambda x: x[0].upper()+x[1:]) 
    
    fig = px.bar(df_validator[["timestamp", "per", "validator"]], x="timestamp", 
                  y="per", 
                  color="validator", 
                  color_discrete_sequence=colors_val
                 )
   
    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Slots per Hour (last 7 days)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="% of total slots",
        legend_title="Validators",
       
        #title_xanchor="left",
        #title_yanchor="auto",
        
        #margin={"l":100, "t":100},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        #paper_bgcolor='#eee',
        plot_bgcolor="rgba(255,255,255,0.9)",

        #yaxis=dict(fixedrange =True),
        autosize=True, 
        #height=580,
        #width=1600,
        height=580,
       
        
    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(hovertemplate="<b>%{text}: %{y:.0f} %<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16)))
    #fig.update_traces(width=1000)
    #fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    
    fig.update_layout(bargap=0.1,
                      bargroupgap = 0.01,
                     )
    fig.update_yaxes(title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.2)
        if i.legendgroup == "Other/None":
            #i.fillpattern["size"]= 5
            pass
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
    return fig
create_total_lido_validator_share_per_hour_chart_chart = create_total_lido_validator_share_per_hour_chart()
if JUPYTER:
    create_total_lido_validator_share_per_hour_chart_chart.show()

In [ ]:
DS = "ethereum-data-nero.eth.3_builders_over_time"
query = build_query("timestamp, builder, slot", DS, "ORDER BY slot DESC")
df_builder = pd.read_gbq(query)
def create_total_builder_share_chart():
    print("start create_total_builder_share_chart")
    df = df_builder
    #df = #df[df["slot"]>10]    
    df = df.set_index(["timestamp","builder"]).reset_index()
    _df = df[df["timestamp"] == max(df["timestamp"])]
    _df = _df.sort_values("slot", ascending=False)["builder"].tolist()
    _df.remove("Vanilla Builders")
    _df.remove("Flashbots")
    df = df.set_index('builder')
    df = df.loc[["Vanilla Builders", "Flashbots"] + _df].reset_index()
    df = df[df["timestamp"] != max(df["timestamp"])]
    time_axis = df["timestamp"].drop_duplicates().values.tolist()[::int(len(df["timestamp"].drop_duplicates())/10)][:-1]+[df["timestamp"].drop_duplicates().iloc[-1]]
    df["builder"] = df["builder"].apply(lambda x: x[0:12] if x.startswith("0x") else x[0:17])
    fdf = pd.DataFrame()
    ddf = pd.DataFrame(df[df["builder"]=="Vanilla Builders"]["timestamp"])
    for i in df["builder"].drop_duplicates():
        _df = df[df["builder"] == i]
        _df = pd.merge(ddf, _df, on="timestamp", how="left").fillna(0)
        _df.builder = i
        fdf = pd.concat([fdf, _df], ignore_index=True)
    patternmap = [(i, None) for i in fdf["builder"].drop_duplicates()]
    patternmap.remove(('Vanilla Builders', None))
    patternmap.append(('Vanilla Builders', "+"))
    fig = px.area(fdf, x="timestamp", 
                  y="slot", 
                  color="builder", 
                  line_group="builder",
                  groupnorm="percent",
                  color_discrete_sequence=px.colors.qualitative.Plotly,
                  pattern_shape="builder", pattern_shape_map=dict(patternmap),
                  text="builder"
                 )
    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Total Slot Share (cumulative)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="% of total slots",
        legend_title="Builders",
       
        #title_xanchor="left",
        #title_yanchor="auto",        
        margin={"l":100, "t":100},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        paper_bgcolor='#eee',
        #yaxis=dict(fixedrange =True),
        autosize=True, 
        #height=580,
        #width=700,
        height=580,
        xaxis=dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = BLACK,
                rangeselector=dict(
                    buttons=list([
                        dict(count=7,
                             label="7d",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        #dict(count=6,
                        #     label="6m",
                        #     step="month",
                        #     stepmode="backward"),
                        dict(step="all", label="since merge")
                    ])
                ),
                
            type="date"
        )
        
    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(mode='lines', line=dict(width=0.8))
    fig.update_traces(hoverinfo="skip",hovertemplate="<b>%{x}<br>%{text}: %{y:.0f} %<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16)))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2), range=[0,100],
                    title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.2)
        if i.legendgroup == "Vanilla Builders":
            i.fillpattern["size"]= 5
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
    return fig
create_total_builder_share_chart_chart = create_total_builder_share_chart()
if JUPYTER:
    create_total_builder_share_chart_chart.show()

In [ ]:
def create_total_builder_share_per_hour_chart():
    print("start create_total_builder_share_per_hour_chart")
    DS = "ethereum-data-nero.eth.3_builders_over_time_last_7_days"
    query = build_query("timestamp, builder, slot", DS, "ORDER BY slot desc")
    df_builder = pd.read_gbq(query)
    df_builder = df_builder[df_builder["builder"].isin(df_builder["builder"].drop_duplicates().head(16).tolist())]
    df_builder = pd.merge(df_builder, df_builder[["timestamp", "slot"]].groupby("timestamp").sum().reset_index(), on="timestamp", how="right")
    df_builder["per"] = df_builder["slot_x"] / df_builder["slot_y"]*100
    df_builder = df_builder.rename(columns={"slot_x": "slot"})
    df_builder = df_builder[~df_builder["timestamp"].isin(df_builder.sort_values("timestamp")["timestamp"].drop_duplicates().tail(2))]
    df_builder["builder"] = df_builder["builder"].apply(lambda x: x[0:12] if x.startswith("0x") else x[0:17])
    _df = df_builder.sort_values("slot", ascending=False)["builder"].drop_duplicates().tolist()
    _df.remove("Vanilla Builders")
    df_builder = df_builder.set_index('builder')
    df_builder = df_builder.loc[["Vanilla Builders"] + _df].reset_index()
    max_entry = df_builder[df_builder["timestamp"] == max(df_builder["timestamp"])]
    if sum(max_entry[max_entry["builder"] == "Vanilla Builders"]["per"]) / 2 > sum(max_entry[max_entry["builder"] != "Vanilla Builders"]["per"]):
        df_builder = df_builder[df_builder["timestamp"] != max(df_builder["timestamp"])]

    fig = px.bar(df_builder[["timestamp", "per", "builder"]], x="timestamp", 
                  y="per", 
                  color="builder", 
                  color_discrete_sequence=[hextorgba(j[1:], 0.8) for j in colors],
                  text="builder",
                )
   
    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Slots per Hour (last 7 days)</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="% of total slots",
        legend_title="Builders",
       
        #title_xanchor="left",
        #title_yanchor="auto",
        
        margin={"l":100, "t":70},
        font=dict(
            family="Courier New, monospace",
            size=18,  # Set the font size here
            color=BLACK
        ),
        #paper_bgcolor='#eee',
        plot_bgcolor="rgba(255,255,255,0.9)",
        autosize=True, 
        #height=580,
        #width=1600,
        height=580,
    
        #)
        
    )
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.update_traces(hovertemplate="<b>%{text}: %{y:.0f} %<extra></extra><b>",
                      hoverlabel=dict(font=dict(color=BLACK, size=16)),textposition = "none")
    #fig.update_traces(width=1000)
    fig.update_yaxes(title_font=dict(size=20))
    
    fig.update_layout(bargap=0.1,
                      bargroupgap = 0.01,
                     )
    #fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2), range=[0,100])
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.2)
        if i.legendgroup == "Vanilla Builders":
            #i.fillpattern["size"]= 5
            pass
            #i.fillpattern["fgcolor"] = BLACK_ALPHA.format(0.2)
    return fig
create_total_builder_share_per_hour_chart_chart = create_total_builder_share_per_hour_chart()
if JUPYTER:
    create_total_builder_share_per_hour_chart_chart.show()

In [ ]:
def exchange_with_known_builder(hc):
    if "Illuminate Dmocratize Dstribute" in hc:
        hc = "Flashbots"
    if "bloXroute" in hc:
        hc = "BloXroute"
    if "Blocknative" in hc:
        hc = "Blocknative"
    if "builder0x69" in hc:
        hc = "builder0x69"
    if "Manifold" in hc:
        hc = "Manifold" 
    return hc.replace("https://", "")


def create_pie_charts():
    print("start create_pie_charts")
    _colors = colors[::-1]
    _colors.remove("#8c564b")
    _colors.remove("#7f7f7f")
    _colors.remove("#2ca02c")
    _colors.remove("#d62728")
    _colors.remove("#9467bd")

    DS = "ethereum-data-nero.eth.2_builder_pie"
    query = build_query("builder, slot", DS, "ORDER BY slot DESC")
    df = pd.read_gbq(query).dropna()
    df["builder"]= df["builder"].apply(lambda x: exchange_with_known_builder(x))
    df["builder"]= df["builder"].apply(lambda x: x[0:15] if x != "Vanilla Builders" else x)
    df = df.groupby("builder")["slot"].sum().reset_index()
    
    #df = pd.read_csv(FOLDER + "pie_total.csv").sort_values("slot", ascending=True)
    colorsmap = dict([(i,hextorgba(j[1:], 0.7)) for i,j in zip(df.sort_values("slot", ascending=False)["builder"].tolist(), colors[::-1])])

    df2 = df[df["builder"] != "Vanilla Builders"]
    #df2 = pd.read_csv(FOLDER + "pie_mevboost.csv").sort_values("block_hash", ascending=True)
    fig = make_subplots(rows=1, cols=2,  
                        subplot_titles=("Total block market\n", "MEV-Boost block market\n"), 
                        specs=[[{"type": "pie"}, {"type": "pie"}]])

    figure1 = px.pie(df, values='slot', names='builder', hole=.3,
                 hover_data=['slot'],color="builder",  color_discrete_map=colorsmap)
    if "Vanilla Builders" in colorsmap.keys():
        del colorsmap["Vanilla Builders"]

    figure2 = px.pie(df2, values='slot', names='builder', hole=.3,
                 hover_data=['slot'],color="builder", color_discrete_map=colorsmap)
    

    for i, figure in enumerate([figure1,figure2]):
        for trace in range(len(figure["data"])):
            fig.append_trace(figure["data"][trace], row=1, col=i+1)

    fig.update_layout(
        uniformtext_minsize=20, 
        uniformtext_mode='hide',
        title=dict(
            text='<span style="font-size: 24px;">MEV-Boost <b>Builder</b> Market <span style="font-size:18px;">(last 14 days)</span></span></br> ',
            font=dict(
                family="Courier New, monospace",
                size=30,
                color=BLACK
            )),
        font=dict(
                family="Courier New, monospace",
                size=18,  # Set the font size here
                color=BLACK
            ),
        height=510
    )
    fig.update_traces(hovertemplate="<b>%{label}:<br>%{value:,.0f} blocks</b>",hoverinfo='label+value',
                      textposition='inside',texttemplate = "<b>%{percent}<b>", 
                      textfont_size=14, hoverlabel=dict(font=dict(color=BLACK, size=16)),
                      marker=dict(line=dict(color=BLACK, width=2)), 
                      textfont=dict(color=BLACK))
    
    fig.layout.annotations[1].update(y=1.01)
    fig.layout.annotations[0].update(y=1.01)
    fig.update_annotations(font_size=22)

    return fig

create_pie_charts_chart = create_pie_charts()
if JUPYTER:
    create_pie_charts_chart.show()

In [ ]:
def create_pie_relay_charts():
    print("start create_pie_relay_charts")
    #df = pd.read_csv(FOLDER + "pie_total_relay.csv").sort_values("slot", ascending=False)
    DS = "ethereum-data-nero.eth.2_relay_pie"
    query = build_query("relay, slot", DS, "ORDER BY slot DESC")
    df = pd.read_gbq(query)
    def shorten(x):
        if "bloxroute" in x:
            i, j = x.split("(")
            j = f'<span style="font-size:16px;">({j}</span>'
            return i + j
        else:
            return x

    df["relay"] = df["relay"].apply(shorten)
    df["relay"] = df["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    df = df.replace("Ultrasound", "Ultra Sound")
    colorsmap = dict([(i,hextorgba(j[1:], 0.7)) for i,j in zip(df["relay"].tolist(), colors[::-1])])
    df2 = df[df["relay"] != "Other/None"]

    df2["relay"] = df2["relay"].apply(shorten)
    fig = make_subplots(rows=1, cols=2,  subplot_titles=("Total block market\n", "MEV-Boost block market\n"), specs=[[{"type": "pie"}, {"type": "pie"}]])

    figure1 = px.pie(df, values='slot', names='relay', hole=.3,
                 hover_data=['slot'], color="relay", color_discrete_map=colorsmap)# color=px.colors.qualitative.Plotly[0:-1][::-1])
    del colorsmap["Other/None"]
    figure2 = px.pie(df2, values='slot', names='relay', hole=.3,
                 hover_data=['slot'], color="relay",  color_discrete_map=colorsmap)
    
    for i, figure in enumerate([figure1,figure2]):
        for trace in range(len(figure["data"])):
            fig.append_trace(figure["data"][trace], row=1, col=i+1)

    fig.update_layout(
        uniformtext_minsize=20, 
        uniformtext_mode='hide',
        title=dict(
            text='<span style="font-size: 24px;">MEV-Boost <b>Relay</b> Market <span style="font-size:18px;">(last 14 days)</span></span></br> ',
            font=dict(
                family="Courier New, monospace",
                size=30,
                color=BLACK
            )),
        font=dict(
                family="Courier New, monospace",
                size=18,  
                color=BLACK
            ),
        paper_bgcolor='rgba(255, 255, 255, 0.5)', 
        height=510
    )

    fig.update_traces(hovertemplate="<b>%{label}:<br>%{value:,.0f} blocks</b>",hoverinfo='label+value',
                      textposition='inside',texttemplate = "<b>%{percent}<b>",
                      textfont_size=14, hoverlabel=dict(font=dict(color=BLACK, size=16)),
                      marker=dict(line=dict(color=BLACK, width=2)), textfont=dict(color=BLACK),
                     )
    fig.layout.annotations[1].update(y=1.01)
    fig.layout.annotations[0].update(y=1.01)
    fig.update_annotations(font_size=18)
    return fig

create_pie_relay_charts_chart = create_pie_relay_charts()
if JUPYTER:
    create_pie_relay_charts_chart.show()

In [ ]:
def create_pie_validator_charts():
    print("start create_pie_validator_charts")
    #df = pd.read_csv(FOLDER + "pie_total_relay.csv").sort_values("slot", ascending=False)
    DS = "ethereum-data-nero.eth.2_validator_pie"
    query = build_query("validator, slot", DS, "ORDER BY slot DESC")
    df = pd.read_gbq(query)

    df["validator"] = df["validator"].apply(lambda x: x[0].upper()+x[1:15]) 
    colorsmap = dict([(i,hextorgba(j[1:], 0.7)) for i,j in zip(df["validator"].tolist(), colors[::-1])])
    #df2 = df[df["relay"] != "Other/None"]

    #df2["validator"] = df2["validator"].apply(shorten)
    #fig = make_subplots(rows=1, cols=2,  subplot_titles=("Total block market\n", "MEV-Boost block market\n"), specs=[[{"type": "pie"}, {"type": "pie"}]])

    fig = px.pie(df, values='slot', names='validator', hole=.3,
                 hover_data=['slot'], color="validator", color_discrete_map=colorsmap)# color=px.colors.qualitative.Plotly[0:-1][::-1])
    #del colorsmap["Other/None"]
    #figure2 = px.pie(df2, values='slot', names='validator', hole=.3,
    #             hover_data=['slot'], color="validator",  color_discrete_map=colorsmap)
    
    #for i, figure in enumerate([figure1,figure2]):
    #    for trace in range(len(figure["data"])):
    #        fig.append_trace(figure["data"][trace], row=1, col=i+1)

    fig.update_layout(
        margin={"l":20, "t":120, "r": 20, "b": 20},
        uniformtext_minsize=20, 
        uniformtext_mode='hide',
        legend=dict(
                bgcolor="rgba(255,255,255,0.0)",
                yanchor="middle",
                y=0.6,
                xanchor="center",
                x=1,
                font=dict(
                    family="Courier New, monospace",
                    size=18,  
                    color=BLACK
                ),
        ),
        title=dict(
            pad = {"b": 150, "t": 20},
            text='<span style="font-size: 24px;">Total <b>Validator</b> Market</span></br><span style="font-size:18px;">(last 14 days)</span><br> ',
            xanchor="center",
            x=0.45,
            font=dict(
                family="Courier New, monospace",
                size=30,
                color=BLACK
            )),
        font=dict(
                family="Courier New, monospace",
                size=18,  
                color=BLACK
            ),
        paper_bgcolor='rgba(255, 255, 255, 0.5)', 
        height=510
    )

    fig.update_traces(hovertemplate="<b>%{label}:<br>%{value:,.0f} blocks</b>",hoverinfo='label+value',
                      textposition='inside',texttemplate = "<b>%{percent}<b>",
                      textfont_size=24, hoverlabel=dict(font=dict(color=BLACK, size=16)),
                      marker=dict(line=dict(color=BLACK, width=2)), textfont=dict(color=BLACK),
                     )
    #fig.layout.annotations[1].update(y=1.01)
    fig.update_annotations(font_size=22)
    return fig

create_pie_validator_charts_chart = create_pie_validator_charts()
if JUPYTER:
    create_pie_validator_charts_chart.show()

In [ ]:
def create_pie_total_charts():
    print("start create_pie_total_charts")
    #df = pd.read_csv(FOLDER + "pie_total_relay.csv").sort_values("slot", ascending=False)
    DS = "ethereum-data-nero.eth.2_relay_pie"
    query = build_query("relay, slot", DS, "ORDER BY slot DESC")
    df = pd.read_gbq(query)
    df = df.replace("Other/None", "Vanilla Builders")
    df_copy = df[df["relay"] == "Vanilla Builders"]
    df_copy["relay"] = "Vanilla Builders"
    df_copy.loc[len(df_copy), ("relay", "slot")] = "MEV-Boost", df[df["relay"] != "Vanilla Builders"]["slot"].sum()
    df = df_copy
    fig = px.pie(df, values='slot', names='relay', hole=.3,
                 hover_data=['slot'], color="relay",  
                 color_discrete_map={'Vanilla Builders': hextorgba('b1d7e5', 0.8), 'MEV-Boost': hextorgba('ffbb78', 0.8)})



    fig.update_layout(
        uniformtext_minsize=18, 
        uniformtext_mode='hide',
        legend=dict(
                bgcolor="rgba(255,255,255,0.0)",
                yanchor="bottom",
                y=-0.5,
                xanchor="center",
                x=0.5,
                font=dict(
                    family="Courier New, monospace",
                    size=18,  
                    color=BLACK
                ),
        ),
        margin={"l":20, "t":80, "r": 20, "b": 20},
        title=dict(
            text='<span style="font-size: 24px;font-weight:bold;">MEV-Boost Slot Share</br><span style="font-size:14px;">(last 14 days)</span><br> </span>',
            xanchor="center",
            x=0.5,
            yanchor="middle",
            y=0.9,
            pad = {"b": 150, "t": 20},
            font=dict(
                family="Courier New, monospace",
                size=18,
                color=BLACK
            )),
        font=dict(
                family="Courier New, monospace",
                size=18,  
                color=BLACK
            ),
        paper_bgcolor='rgba(255, 255, 255, 0.5)', 
        height=400
    )

    fig.update_traces(hovertemplate="<b>%{label}:<br>%{value:,.0f} blocks</b>",hoverinfo='label+value',
                      textposition='inside',texttemplate = "<b>%{percent}<b>",
                      textfont_size=14, hoverlabel=dict(font=dict(color=BLACK, size=16)),
                      marker=dict(line=dict(color=BLACK, width=2)), textfont=dict(color=BLACK),
                     )
    #fig.layout.annotations[1].update(y=1.01)
    #fig.layout.annotations[0].update(y=1.01)
    fig.update_annotations(font_size=22)
    return fig

create_pie_total_charts_chart = create_pie_total_charts()
if JUPYTER:
    create_pie_total_charts_chart.show()

In [ ]:
def nr_of_validators(df2):
    count_vals = set()
    for ix, i in df2.iterrows():
        if ix < nrofrelayers - 1:
            continue
        if i["node"].startswith("0x"):
            break
        count_vals.add(i["label"])
    return len(count_vals)

def exchange_with_known_builder(hc):
    
    if hc == None:
        return "anon"

    if "Illuminate Dmocratize Dstribute" in hc:
        hc = "Flashbots"
    elif "bloXroute" in hc:
        hc = "BloXroute"
    elif "Blocknative" in hc:
        hc = "Blocknative"
    elif "builder0x69" in hc:
        hc = "builder0x69"
    elif "Manifold" in hc:
        hc = "Manifold"  
    
    if hc.startswith("0x") and len(hc) > 30:
        hc = "anon"
        
    if len(hc) > 11:
        hc = hc[0:11]
    return hc

def sankey_builders(x):
    return x["node"][0:8]+"... (" + exchange_with_known_builder(x["label"])+ ")"

def create_sankey():
    print("start create_sankey")
    span = '<span style="font-size: 24px;font-weight:bold;">{}</span>'
    DS = "ethereum-data-nero.eth.7_recent_builders_1w"
    query = build_query("node, label", DS, "ORDER BY label DESC")
    df6 = pd.read_gbq(query).dropna()
    df6 = df6.replace("Ultrasound", "Ultra Sound")
    DS = "ethereum-data-nero.eth.7_recent_builders"
    query = build_query("node, label", DS, "ORDER BY label DESC")
    df7 = pd.read_gbq(query).dropna()
    df7 = df7.replace("Ultrasound", "Ultra Sound")
    
    DS = "ethereum-data-nero.eth.sankey_final_val"
    query = build_query("source, target, link_count, link_color", DS, "ORDER BY link_count DESC")
    df = pd.read_gbq(query)
    df = df.rename(columns={"link_count": "value"})
    df["link_color"] = df["link_color"].apply(lambda x: hextorgba(x.replace("#", "")))

    DS = "ethereum-data-nero.eth.sankey_node_ids_val"
    query = build_query("id, node, color, label", DS, "ORDER BY id ASC")
    df2 = pd.read_gbq(query).dropna()
    
    nrofvals = nr_of_validators(df2)
    df2["node"].iloc[nrofvals+nrofrelayers-1:] = df2.iloc[nrofvals+nrofrelayers-1:].apply(lambda x: sankey_builders(x), axis=1)
    df7["node"] = df7.apply(lambda x: sankey_builders(x), axis=1)
    df6["node"] = df6.apply(lambda x: sankey_builders(x), axis=1)
    
    df2["node"].iloc[0:nrofrelayers] = df2.iloc[0:nrofrelayers].apply(lambda x: (x["node"][0].upper()+x["node"][1:]).replace("(", "<br>("), axis=1)
    df2["node"].iloc[nrofrelayers:nrofrelayers+nrofvals-1] = df2.iloc[nrofrelayers:nrofrelayers+nrofvals-1].apply(lambda x: x["node"][0].upper()+x["node"][1:], axis=1)
    df2 = df2.replace("Ultrasound", "Ultra Sound")
    
    df2 = df2.set_index("id")
        
    smallbuilders = df[["source", "value"]].groupby("source").sum().reset_index()
    smallbuilders = smallbuilders[smallbuilders["value"] < 800]
    smallbuilders = [df2.iloc[i]["node"] for i in smallbuilders.source]

    fig = go.Figure(data=[go.Sankey(
        textfont=go.sankey.Textfont(size=16, color="black", family="Courier New"),
        node = dict(
          pad = 20,
          thickness = 20,
          line = dict(color = BLACK, width = 0.5),
          label = df2["node"].tolist(),
          color = df2["color"],
          hovertemplate='<b>%{label}<b><br />%{value} blocks',
        ),
        link = dict(
          source = df["source"],
          target = df["target"],
          value = df["value"],
        color = df["link_color"],
        hovertemplate='<b>%{source.value:,.0f} blocks<br /><b>',  
        )
        ,hoverlabel=dict(font=dict(size=16)),
    
    
    )])
    
    node = fig.data[0].node
    link = fig.data[0].link
    _node = copy.copy(fig.data[0].node)
    _link = copy.copy(fig.data[0].link)
    
    def get_node_for(relay, fig, smallb=False):
        node = fig.data[0].node
        link = fig.data[0].link
        _node = copy.copy(fig.data[0].node)
        _link = copy.copy(fig.data[0].link)

        lab = copy.copy(list(fig.data[0].node.label))

        col = copy.copy(list(fig.data[0].node.color))

        _source = copy.copy(list(fig.data[0].link["source"]))
        _target = copy.copy(list(fig.data[0].link["target"]))
        _value = copy.copy(list(fig.data[0].link["value"]))
        _color = copy.copy(list(fig.data[0].link["color"]))

        if not smallb:
            out = []
            for ix, i in enumerate(lab):
                
                if not relay in i:
                    out.append(ix) 
                if "(Flashbots)" in i:
                    out.append(ix)
                if "(viva ultrasound" in i:
                    out.append(ix)
        else:
            if smallb == "smallb":
                smallbuilders = df[["source", "value"]].groupby("source").sum().reset_index()
                smallbuilders = smallbuilders[smallbuilders["value"] < 1500]
                out = [i for i in smallbuilders.source]
            elif smallb == "last30":
                out = []
                for ix, i in enumerate(lab):
                    if i not in df7["node"].tolist() and i.startswith("0x"):
                        out.append(ix)
            elif smallb == "last7":
                out = []
                for ix, i in enumerate(lab):
                    if i not in df6["node"].tolist() and i.startswith("0x"):
                        out.append(ix)
            
        out_link = []
        if not smallb:
            for ix,(i, j) in enumerate(zip(_source, _target)):
                if i in out and j in out:
                    out_link.append(ix)
            
        else:
            for ix,(i, j) in enumerate(zip(_source, _target)):
                if i in out:
                    out_link.append(ix)
                
        for ix in sorted(out_link)[::-1]:
            del _source[ix]
            del _target[ix]
            del _value[ix] 
            del _color[ix]
        
        _node.label = tuple(lab)
        _node.color = tuple(col)

        _link.source = tuple(_source)
        _link.target = tuple(_target)
        _link.value = tuple(_value)
        _link.color = tuple(_color)
        return (_node, _link)
    
    relays = {}
    for i in fig.data[0].node.label[:11]:
        relays[i] = get_node_for(i, fig)
        
    validators = {}
    for i in ["Lido", "Kraken", "Coinbase", "Binance", "Staked.us", "Stakefish"]:
        validators[i] = get_node_for(i, fig)
    
    hidesmall = get_node_for("", fig, "smallb")
    last30 = get_node_for("", fig, "last30")
    last7 = get_node_for("", fig, "last7")
    
    
    def exchange_bloxroute(x):
        if "Bloxroute" in x:
            if "max profit" in x:
                return "Bloxroute mp."
            elif "ethical" in x:
                return "Bloxroute et."
            elif "regulated" in x:
                return "Bloxroute rgl."
        return x

    updatemenus=[
        dict(
            bgcolor = "#ffffff",
            #type="buttons",
            pad={"r": 0, "t": 60},
            showactive=True,
            x=1,
            xanchor="right",
            y=1.1,
            yanchor="top",
            buttons=list(
                [dict(label="All",
                     method="update",
                     args=[{"visible": [True],
                            "node":_node,
                            "link": _link},
                           {"title": span.format("Block Flow (MEV-Boost)"),
                             "height":2200,
                            "annotations": []}])] + 
                [dict(label="Hide small builders",
                     method="update",
                     args=[{"visible": [True],
                            "node":hidesmall[0],
                            "link": hidesmall[1]},
                           {"title": span.format("Block Flow (MEV-Boost)"),
                             "height":1200,
                            "annotations": []}])]
               + [dict(label="Active builders (30 days)",
                    method="update",
                    args=[{"visible": [True],
                           "node":last30[0],
                           "link": last30[1]},
                          {"title": span.format("Block Flow (MEV-Boost)"),
                            "height":1500,
                           "annotations": []}])]
                + [dict(label="Active builders (7 days)",
                    method="update",
                    args=[{"visible": [True],
                           "node":last7[0],
                           "link": last7[1]},
                          {"title": span.format("Block Flow (MEV-Boost)"),
                            "height":1400,
                           "annotations": []}])] +
                
                [dict(label="{}".format(exchange_bloxroute(i)),
                     method="update",
                     args=[{"visible": [True],
                            "node":relays[i][0],
                            "link": relays[i][1]},
                           {"title": span.format("{} Relay".format(i)), "height":1200 if i != "Flashbots" else 2000,
                            "annotations": []}]
                    ) for i in fig.data[0].node.label[:11]] + 
                
                [dict(label="{}".format(i),
                     method="update",
                     args=[{"visible": [True],
                            "node":validators[i][0],
                            "link": validators[i][1]},
                           {"title": span.format("{} Relay".format(i)), "height":1200 if i != "Flashbots" else 2000,
                            "annotations": []}]
                    ) for i in ["Lido", "Kraken", "Coinbase", "Binance", "Staked.us", "Stakefish"]]  
            )
        )
    ]
      

    fig.update_layout(title_text=span.format("Block Flow (MEV-Boost)"),
                      font_size=16,
                      paper_bgcolor='#eee',
                      font=dict(
                                family="Courier New, monospace",
                                #size=24,  # Set the font size here
                                color=BLACK
                               ), autosize=False, width=800, height=2200,
                      updatemenus=updatemenus)
                      
    return fig
create_sankey_chart = create_sankey()
if JUPYTER:
    create_sankey_chart.show()

In [ ]:
def create_avg_proposer_payment():
    print("start create_avg_proposer_payment")
    MAX = 0.65
    MIN = 0
    def my_mean(s):
        outlier_count = ((s<MIN) | (s > MAX)).sum()
        if outlier_count > 2: # defined 2 as the threshold - can put any other number here
            return np.NaN
        res =  s[(s <= MAX) & (s >= MIN)].mean()
        return res

    fig = go.Figure()
    #df = pd.read_csv(FOLDER + "avg_proposer_val_gas.csv")
    DS = "ethereum-data-nero.eth.5_avg_value_relay"
    query = build_query("timestamp, relay, value", DS, "ORDER BY timestamp ASC")
    df = pd.read_gbq(query)
    df["relay"] = df["relay"].apply(lambda x: x.replace("(", "<br>("))
    df["value"] = df["value"].astype(float)
    #df["timestamp"] = df["timestamp"].apply(lambda x: datetime.strftime(datetime.strptime(x, "%Y-%m-%d"), "%Y-%W"))
    #df["timestamp"] = df["timestamp"].apply(lambda x: datetime.strftime(datetime.strptime(x+ '-1', "%Y-%W-%w"), "%Y-%m-%d"))
    df["relay"] = df["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    for ix, relay in enumerate(["Flashbots", "Bloxroute <br>(max profit)", "Bloxroute <br>(ethical)", 
                  "Bloxroute <br>(regulated)", "Manifold", "Eden", "Blocknative", "Relayooor",
                  "Ultrasound", "Agnostic Gnosis",  "Aestus"]):
        if relay in ["Flashbots", "Bloxroute <br>(max profit)"]:
            show = True
        else:
            show = "legendonly"
        _df = df[df["relay"] == relay]#.groupby("timestamp").mean().reset_index()
        if ix >= 7:
            ix = ix+3
        if ix >= 11:
            ix += 1
        if relay in ["Ultrasound"]:
            _name = "Ultra Sound"
        else:
            _name = relay
        fig.add_scatter(mode="lines+markers",
                        x=_df["timestamp"], 
                        y=_df["value"], 
                        name = _name,
                        marker=dict(
                            
                            size=_df["value"].apply(lambda x: 10+(x-min(df["value"]))/(max(df["value"])-min(df["value"]))*10),
                        ),
                     visible = show,
                     marker_color=px.colors.qualitative.Plotly[ix],
                     )

    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Avg. MEV-Boost Proposer Payment</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="ETH",
        legend_title="Relay Provider",
        margin={"l":100, "t":100},
        hovermode="closest",
        yaxis_tickformat = "0.1r",
        yaxis = dict(
            type="log",
            tickmode = 'array',
            tickvals = [0.02, 0.06, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1],
        ),
        font=dict(
            family="Courier New, monospace",
            size=18,
            color=BLACK
        ),
        #autosize=True, 
        height=630,
        paper_bgcolor='#eee',
        plot_bgcolor="rgba(255,255,255,0.9)",  
        xaxis = dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = WHITE,
                rangeselector=dict(
                    buttons=list([
                        dict(count=14,
                             label="14d",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(count=6,
                             label="6m",
                             step="month",
                             stepmode="backward"),
                        dict(step="all", label="since merge")
                    ])
                ),
             
            type="date"
        ),
    )
    fig.update_traces(hovertemplate = '<b>Avg. payment: %{y:.2f} ETH</b>', hoverlabel=dict(font=dict(color=BLACK, size=16)))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                     title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.2)
    return fig
    
create_avg_proposer_payment_chart = create_avg_proposer_payment()
if JUPYTER:
    create_avg_proposer_payment_chart.show()

In [ ]:
def create_avg_proposer_payment_per_builder():
    print("start create_avg_proposer_payment_per_builder")
    MAX = 0.65
    MIN = 0
    def my_mean(s):
        outlier_count = ((s<MIN) | (s > MAX)).sum()
        if outlier_count > 2: # defined 2 as the threshold - can put any other number here
            return np.NaN
        res =  s[(s <= MAX) & (s >= MIN)].mean()
        return res

    fig = go.Figure()

    DS = "ethereum-data-nero.eth.5_avg_value_builder"
    query = build_query("timestamp, builder, value", DS, "ORDER BY timestamp ASC, value DESC")
    df = pd.read_gbq(query)
    df["builder"] = df["builder"].apply(lambda x: x.replace("https://", ""))
    df["builder"] = df["builder"].apply(lambda x: x[:13])
    df["value"] = df["value"].astype(float)
    df = df.set_index("builder")
    _df = df_builder[df_builder["builder"] != "Vanilla Builders"]
    _df["builder"] = _df["builder"].apply(lambda x: x.replace("https://", ""))
    _df["builder"] = _df["builder"].apply(lambda x: x[:13])
    
    sorter = _df.groupby("builder")["slot"].sum().reset_index().sort_values("slot", ascending=False)["builder"].tolist()
    df = df.loc[sorter].reset_index()
    df = df[df["builder"].isin(_df["builder"].drop_duplicates().tolist()[:15])]

    for ix, builder in enumerate(df["builder"].drop_duplicates()):
        if builder in ["Flashbots", "BloXroute"]:
            show = True
        else:
            show = "legendonly"
        _df = df[df["builder"] == builder]#.groupby("timestamp").mean().reset_index()
        if ix >= 7:
            ix = ix+3
        if ix >= 11:
            ix += 1
        fig.add_scatter(mode="lines+markers",
                        x=_df["timestamp"], 
                        y=_df["value"], 
                        name = builder,
                        marker=dict(
                            
                            size=_df["value"].apply(lambda x: 10+(x-min(df["value"]))/(max(df["value"])-min(df["value"]))*10),
                        ),
                     visible = show,
                     marker_color=(colors*2)[ix],
                     )

    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Avg. MEV-Boost Payment per block</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title='ETH',
        legend_title="Builder",
        margin={"l":100, "t":100},
        hovermode="closest",
        yaxis_tickformat = "0.1r",
        yaxis = dict(
            type="log",
            tickmode = 'array',
            tickvals = [0.02, 0.06, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1],
        ),
        font=dict(
            family="Courier New, monospace",
            size=18,
            color=BLACK
        ),
        #autosize=True, 
        height=500,
        paper_bgcolor='#eee',
        plot_bgcolor='rgb(255,255,255)',
        xaxis = dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = BLACK,
                rangeselector=dict(
                    buttons=list([
                        dict(count=14,
                             label="14d",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(count=6,
                             label="6m",
                             step="month",
                             stepmode="backward"),
                        dict(step="all", label="since merge")
                    ])
                ),
             
            type="date"
        ),
    )
    fig.update_traces(hovertemplate = '<b>Avg. payment: %{y:.2f} ETH</b>', hoverlabel=dict(font=dict(color=BLACK, size=16)))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                    title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.2)
    return fig
    
create_avg_proposer_payment_per_builder_chart = create_avg_proposer_payment_per_builder()
if JUPYTER:
    create_avg_proposer_payment_per_builder_chart.show()

In [ ]:
def create_avg_proposer_payment_per_validator():
    print("start create_avg_proposer_payment_per_validator")
    MAX = 0.65
    MIN = 0
    def my_mean(s):
        outlier_count = ((s<MIN) | (s > MAX)).sum()
        if outlier_count > 2: # defined 2 as the threshold - can put any other number here
            return np.NaN
        res =  s[(s <= MAX) & (s >= MIN)].mean()
        return res

    fig = go.Figure()
    #df = pd.read_csv(FOLDER + "avg_proposer_val_gas.csv")
    DS = "ethereum-data-nero.eth.5_avg_value_validator"
    query = build_query("timestamp, validator, value", DS, "ORDER BY timestamp ASC")
    df = pd.read_gbq(query).dropna()
    df["value"] = df["value"].astype(float)
    lido = df[df["validator"] == "lido"]
    rock = df[df["validator"] == "rocketpool"]

    others = df[df["validator"] != "lido"]
    others = others[others["validator"] != "rocketpool"]
    df = pd.concat([lido, rock, others], ignore_index=True)
    #df["timestamp"] = df["timestamp"].apply(lambda x: datetime.strftime(datetime.strptime(x, "%Y-%m-%d"), "%Y-%W"))
    #df["timestamp"] = df["timestamp"].apply(lambda x: datetime.strftime(datetime.strptime(x+ '-1', "%Y-%W-%w"), "%Y-%m-%d"))
    df["validator"] = df["validator"].apply(lambda x: x[0].upper()+x[1:15]) 
    for ix, validator in enumerate(df["validator"].drop_duplicates()):
        if validator in ["Lido", "Rocketpool"]:
            show = True
        else:
            show = "legendonly"
        _df = df[df["validator"] == validator]#.groupby("timestamp").mean().reset_index()
        fig.add_scatter(mode="lines+markers",
                        x=_df["timestamp"], 
                        y=_df["value"], 
                        name = validator,
                        marker=dict(
                            size=_df["value"].apply(lambda x: 10+(x-min(df["value"]))/(max(df["value"])-min(df["value"]))*10),
                        ),
                     visible = show,
                     marker_color=(colors*3)[ix],
                     )

    fig.update_layout(
        title='<span style="font-size: 24px;font-weight:bold;">Avg. MEV-Boost Proposer Payment</span>',
        xaxis_title="",
        xaxis_tickangle=-45,
        yaxis_title="ETH",
        legend_title="Validator",
        margin={"l":100, "t":100},
        hovermode="closest",
        yaxis_tickformat = "0.1r",
        yaxis = dict(
            type="log",
            tickmode = 'array',
            tickvals = [0.02, 0.06, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1],
        ),
        font=dict(
            family="Courier New, monospace",
            size=18,
            color=BLACK
        ),
        #autosize=True, 
        height=500,
        #paper_bgcolor='#eee',
        plot_bgcolor='#eee',
        xaxis = dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = BLACK,
                rangeselector=dict(
                    buttons=list([
                        dict(count=14,
                             label="14d",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(count=6,
                             label="6m",
                             step="month",
                             stepmode="backward"),
                        dict(step="all", label="since merge")
                    ])
                ),
             
            type="date"
        ),
    )
    fig.update_traces(hovertemplate = '<b>Avg. payment: %{y:.2f} ETH</b>', hoverlabel=dict(font=dict(color=BLACK, size=16)))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                     title_font=dict(size=20))
    for ix, i in enumerate(fig.data):
        if ix == 0:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.4)
        else:
            i["hoverlabel"]["bgcolor"] = brighter_color((colors*3)[ix],0.2)
    return fig
    
create_avg_proposer_payment_per_validator_chart = create_avg_proposer_payment_per_validator()
if JUPYTER:
    create_avg_proposer_payment_per_validator_chart.show()

In [ ]:
def create_potential_mev_chart():
    print("start create_potential_mev_chart")
    DS = "ethereum-data-nero.eth.6_potential_mev"
    query = build_query("timestamp, reward, value", DS, "ORDER BY timestamp ASC")
    df = pd.read_gbq(query)
    df["reward"] = df["reward"].astype(float)
    df["value"] = df["value"].astype(float)
    
    txfees, mevboost = df.iloc[-14:][["reward", "value"]].sum().to_list()
    mevboostratio = (mevboost-txfees)/mevboost

    df["delta_vr"] = df["value"] - df["reward"]
    
    df2m = df.tail(60)
    df6m = df.tail(60*3)
    df14 = df.tail(14)
    dfsm = df
    #time_axis = list(df["timestamp"][::int(len(df["timestamp"])/10)][:-1])+[df["timestamp"].iloc[-1]]
    
    fig = go.Figure()
    visible = True
    for _df in [df14, df2m, df6m, dfsm]:
        fig.add_trace(go.Bar(name='Gas fees', x=_df["timestamp"], y=_df["reward"], 
                        marker={"pattern":{"shape":"/"}, "color":hextorgba(px.colors.qualitative.Plotly[0][1:],0.8)},
                       hovertemplate ='<b>%{hovertext} ETH</b>',
                       hovertext = [str(round(i,2)) for i in _df["reward"]],
                       showlegend=True, textposition=None,marker_line_color='rgba(0, 0, 0, 0.2)',visible = visible
                      ))
        fig.add_trace(go.Bar(name='MEV', x=_df["timestamp"], y=_df["delta_vr"], 
                        marker={"pattern":{"shape":"/"}, "color":hextorgba(px.colors.qualitative.Plotly[1][1:],0.6)}
                      ,hovertemplate ='<b>%{hovertext} ETH</b>',
                       hovertext = [str(round(i,2)) for i in _df["delta_vr"]],
                       showlegend=True, textposition=None,marker_line_color='rgba(0, 0, 0, 0.2)',visible = visible
                      ))
        visible = False
    
    
    
    h14 = [True]*2 + [False]*2 + [False]*2 + [False]*2
    h2m = [False]*2 + [True]*2 + [False]*2 + [False]*2
    h6m = [False]*2 + [False]*2 + [True]*2 + [False]*2
    hsm = [False]*2 + [False]*2 + [False]*2 + [True]*2
        
    updatemenus=[
            dict(
                bgcolor = "#ffffff",
                #type="buttons",
                pad={"r": 0, "t": 0},
                showactive=True,
                x=1,
                font_size=12,
                type = 'buttons',
                direction = 'left',
                xanchor="right",
                y=1.2,
                yanchor="top",
                buttons=list(
                    [dict(label='14 days',
                         method="update",
                         args=[{"visible": h14},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 14 days)</span>',
                                "yaxis.range": [0, float(max(df14["value"])) * 1.1],
                                "annotations": []}])]+
                    [dict(label='60 days',
                         method="update",
                         args=[{"visible": h2m},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 60 days)</span>',
                                "yaxis.range": [0, float(max(df2m["value"])) * 1.1],
                                "annotations": []}])]+
                    [dict(label='6 months',
                         method="update",
                         args=[{"visible": h6m},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 6 months)</span>',
                                "yaxis.range": [0, float(max(df6m["value"])) * 1.1],
                                "annotations": []}])]+
                    [dict(label='since merge',
                         method="update",
                         args=[{"visible": hsm},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (since merge)</span>',
                                "yaxis.range": [0, float(max(dfsm["value"])) * 1.1],
                                "annotations": []}])]
                ))]
    
    fig.update_layout(barmode='stack', xaxis_tickangle=-45,
                      title='<span style="font-size: 24px;font-weight:bold;">Daily Gas & MEV Revenue</span>',
            xaxis_title="",
            yaxis_title='ETH',
            #legend_title="Relay Provider",
            #margin={"l":100},
            margin={"l":100, "t":10},
            hovermode="x unified",
           legend=dict(
                bgcolor="rgba(255,255,255,0.5)",
                yanchor="top",
                y=0.98,
                xanchor="right",
                x=0.99,
                font=dict(
                    family="Courier New, monospace",
                    size=22,  
                    color=BLACK
                ),
            ),
            font=dict(
                family="Courier New, monospace",
                size=18,  # Set the font size here
                color=BLACK
            ),
             hoverlabel=dict(
               # bgcolor="rgba(255, 255, 255, 0.1)",
                font_size=16,
                font_family="Courier New, monospace"
            ),
            #paper_bgcolor='#eee',
            plot_bgcolor="#eee",
                      
            #yaxis=dict(fixedrange =True),
            xaxis = dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = "rgba(0, 0, 0, 0.2)",
                type="date",
               ),
        height=500,
        updatemenus=updatemenus
                      
                      
        )
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                     title_font=dict(size=20))
    return mevboostratio, fig
mevboostratio, create_potential_mev_chart_chart = create_potential_mev_chart()
if JUPYTER:
    create_potential_mev_chart_chart.show()

In [ ]:
DS = "ethereum-data-nero.eth.4_avg_mevboost_payments"
query = build_query("timestamp, percentile_25, percentile_50, percentile_75", DS, "ORDER BY timestamp ASC")
df_values = pd.read_gbq(query)

def create_avg_payment_chart():
    print("start create_avg_payment_chart")
    df = df_values
    df["percentile_25"] = df["percentile_25"].astype(float)
    df["percentile_50"] = df["percentile_50"].astype(float)
    df["percentile_75"] = df["percentile_75"].astype(float)
        
    df2m = df.tail(60)
    df6m = df.tail(60*3)
    df14 = df.tail(14)
    dfsm = df
    
    fig = go.Figure()
    visible = True
    for _df in [df14, df2m, df6m, dfsm]:
        
        fig.add_trace(go.Scatter(
            x=_df["timestamp"], y=_df["percentile_50"],
            line_color=px.colors.qualitative.Plotly[0],
            #name='Median MEV-Boost Payment',
            showlegend=False, visible=visible
        ))

        fig.add_trace(go.Scatter(
            x=_df["timestamp"],
            y=_df["percentile_25"],
            fill='tonexty',
            fillcolor='rgba(255, 127, 15, 0.0)',
            line_color='rgba(255, 127, 15, 0.8)',
            showlegend=False,
            name='25% Quantile',mode="lines+markers", visible=visible
        ))
        fig.add_trace(go.Scatter(
            x=_df["timestamp"],
            y=_df["percentile_75"],
            fill='tonexty',
            fillcolor='rgba(255, 127, 15, 0.28)',
            line_color='rgba(255, 127, 15, 0.8)',
            name='+/- 25% Quantile',mode="lines+markers", visible=visible
        ))
        fig.add_trace(go.Scatter(
            x=_df["timestamp"], y=_df["percentile_50"],
            line_color=px.colors.qualitative.Plotly[0],
            name='Median MEV-Boost Payment' , marker=dict(
            size=8 ),line_width=2.5,mode="lines+markers", visible=visible
        ))
        visible = False
    h14 = [True]*4 + [False]*4 + [False]*4 + [False]*4
    h2m = [False]*4 + [True]*4 + [False]*4 + [False]*4
    h6m = [False]*4 + [False]*4 + [True]*4 + [False]*4
    hsm = [False]*4 + [False]*4 + [False]*4 + [True]*4
        
    updatemenus=[
            dict(
                bgcolor = "#ffffff",
                #type="buttons",
                pad={"r": 0, "t": 10},
                showactive=True,
                x=1,
                font_size=12,
                type = 'buttons',
                direction = 'left',
                xanchor="right",
                y=1.2,
                yanchor="top",
                buttons=list(
                    [dict(label='14 days',
                         method="update",
                         args=[{"visible": h14},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 14 days)</span>',
                                "yaxis.range": [0, float(max(df14["percentile_75"])) * 1.1],
                                "annotations": []}])]+
                    [dict(label='60 days',
                         method="update",
                         args=[{"visible": h2m},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 60 days)</span>',
                                "yaxis.range": [0, float(max(df2m["percentile_75"])) * 1.1],
                                "annotations": []}])]+
                    [dict(label='6 months',
                         method="update",
                         args=[{"visible": h6m},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (last 6 months)</span>',
                                "yaxis.range": [0, float(max(df6m["percentile_75"])) * 1.1],
                                "annotations": []}])]+
                    [dict(label='since merge',
                         method="update",
                         args=[{"visible": hsm},
                               {"title": '<span style="font-size: 24px;font-weight:bold;">MEV-Boost Payments (since merge)</span>',
                                "yaxis.range": [0, float(max(dfsm["percentile_75"])) * 1.1],
                                "annotations": []}])]
                ))]
    fig.update_layout(
            title='<span style="font-size: 24px;font-weight:bold;">Avg. MEV-Boost Payments</span>',
            xaxis_title="",
            xaxis_tickangle=-45,
            yaxis_title="ETH",
            margin={"l":100, "t":0},
            legend_title="",
            height=550,
            #hovermode="x unified",
            legend=dict(
                bgcolor="rgba(238,238,238,0.5)",
                yanchor="top",
                y=0.98,
                xanchor="right",
                x=0.99,
                font=dict(
                    family="Courier New, monospace",
                    size=22,  
                    color=BLACK
                ),
            ),
             hoverlabel=dict(
               # bgcolor="rgba(255, 255, 255, 0.1)",
                font_size=16,
                font_family="Courier New, monospace"
            ),
            #pap
            xaxis = dict(
               #gridcolor = BLACK_ALPHA.format(0.5),
               categoryorder = "total descending",
               #tickmode = 'array',
               #tickvals = _df.index[::2],
               #ticktext = [i for i in _df["timestamp"][::2]],
                gridcolor = BLACK,
                type="date",               
           ),
            yaxis = dict(
                gridcolor = BLACK_ALPHA.format(0.5),     
                #fixedrange =True
               ),
            font=dict(
                family="Courier New, monospace",
                size=20,  
                color=BLACK
            ),
            paper_bgcolor='#eee',
            plot_bgcolor="rgba(255,255,255,0.9)",
        updatemenus=updatemenus
        
        
        )
    fig.update_traces(hovertemplate = '<b>%{y:.3f} ETH</b>',
                          hoverlabel=dict(font=dict(color='#ffffff')),
                          marker=dict(line=dict(color="#ffffff", width=2)), 
                          textfont=dict(color="#ffffff"), opacity=0.95)
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                     title_font=dict(size=20))
    return fig
create_avg_payment_chart_chart = create_avg_payment_chart()
if JUPYTER:
    create_avg_payment_chart_chart.show()

In [ ]:
def create_builder_bar_chart():
    print("start create_builder_bar_chart")
    DS = "ethereum-data-nero.eth.4_proposer_payments_n_days"
    query = build_query("builder, value, frame", DS, "ORDER BY frame ASC, value DESC")
    df = pd.read_gbq(query)
    df["value"] = df["value"].astype(float)
    def clean(builderlabel):
        builderlabel =  builderlabel.replace("https://", "")    
        builderlabel = builderlabel.replace("http://", "")
        return builderlabel[0:13]
    df["builder"] = df["builder"].apply(clean)
    df = df.groupby(["builder", "frame"])["value"].sum().reset_index()
    custom_dict = {'Total': 3, 'Last 30 days': 2, 'Last 14 days': 1, "Last 7 days":0} 
    df = df.sort_values("frame", key=lambda x: x.map(custom_dict))
    df = df.rename(columns={'frame': 'time frame'})
    df = df.sort_values("value", ascending=False)
    df = df.sort_values("time frame", key=lambda x: x.map(custom_dict))
    fig = px.histogram(df, x='builder', y='value', color="time frame",  barmode='group',
                      color_discrete_sequence=[hextorgba(j[1:], 0.8) for j in colors[:2]],
                       pattern_shape="time frame", 
                      pattern_shape_sequence=["/", "/"], animation_frame='time frame',
                       category_orders = dict(builder=df[df["time frame"]=="Total"]["builder"].tolist())
                      )
    fig.update_layout(
        xaxis_tickangle=45,
        
            title='<span style="font-size: 24px;font-weight:bold;">Avg. Proposer Payment per Builder <span style="font-size:18px;">(Largest 15 Builders)</span></span>',
            xaxis_title="",
            yaxis_title='ETH',
            margin={"l":100},
            legend_title="",
             hoverlabel=dict(
               # bgcolor="rgba(255, 255, 255, 0.1)",
                font_size=16,
                font_family="Courier New, monospace"
            ),
            #pap
            legend=dict(
                yanchor="top",
                y=0.98,
                xanchor="right",
                x=0.99,
                font=dict(
                    family="Courier New, monospace",
                    size=18,  
                    color=BLACK
                ),
            ),
            xaxis = dict(
               gridcolor = BLACK_ALPHA.format(0.5),
           ),
            yaxis = dict(
                gridcolor = BLACK_ALPHA.format(0.5),     
                range = [0, max(df["value"]) *1.05],
                fixedrange= True 
               ),
            font=dict(
                family="Courier New, monospace",
                size=18,  
                color=BLACK
            ),
        height=550,
        
        paper_bgcolor='#eee',
        plot_bgcolor='#ffffff'
        )
    
    fig.layout.sliders[0].pad.t = 140
    fig.layout.sliders[0].pad.r = 80
    fig.layout.sliders[0].pad.l = 0
    
    fig.layout.sliders[0].currentvalue.prefix = "Time Frame: "    
    fig.layout.sliders[0].currentvalue.font.size = 22
    fig.layout.sliders[0].font.size = 18
    fig["layout"].pop("updatemenus")
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                     title_font=dict(size=20))
    fig.update_traces(hovertemplate = '<b>Avg. payment: %{y:.3f} ETH</b>',textposition='inside', 
                           hoverlabel=dict(font=dict(color="#ffffff", size=16)),
                          marker=dict(line=dict(color="#000000", width=2)), 
                          textfont=dict(color="#ffffff"), opacity=0.95)
    return fig
create_builder_bar_chart_chart = create_builder_bar_chart()
if JUPYTER:
    create_builder_bar_chart_chart.show()

In [ ]:
def create_cumulated_builder():
    print("start create_cumulated_builder")
    DS = "ethereum-data-nero.eth.4_cumulative_builders"
    query = build_query(" builder, timestamp, slot", DS, "ORDER BY builder, slot DESC")
    df = pd.read_gbq(query)
    df = df[df["timestamp"] != max(df["timestamp"])]
    
    def clean(builderlabel):
        builderlabel =  builderlabel.replace("https://", "")    
        builderlabel = builderlabel.replace("http://", "")
        if builderlabel == "Vanilla Builders":
            return "Vanilla<br>Builders"
        return builderlabel[0:13]
    df["builder"] = df["builder"].apply(clean)
    #df["builder"] = df["builder"].apply(lambda x: x[:12])
    
    df = df.groupby(["builder", "timestamp"])["slot"].sum().reset_index().sort_values("timestamp")
    #df = df.rename(columns={'frame': 'time frame'})
    fig = px.histogram(df, x='builder', y='slot',  barmode='group',
                      color_discrete_sequence=[hextorgba(j[1:], 0.8) for j in colors[:2]],
                      pattern_shape_sequence=["/", "/"], animation_frame='timestamp',
                       category_orders = dict(builder=df.sort_values("slot", ascending=False)["builder"].drop_duplicates().tolist())
                      )
    fig.layout.sliders[0]['active'] = len(set(df["timestamp"])) -1
    
    fig.update_layout(
            xaxis_tickangle=45,
            hoverlabel=dict(
               # bgcolor="rgba(255, 255, 255, 0.1)",
                font_size=16,
                font_family="Courier New, monospace"
            ),
            #pap
            title='<span style="font-size: 24px;font-weight:bold;">Blocks per Builder <span style="font-size:18px;">(Largest 10 Builders)</span></span>',
            xaxis_title="",
            yaxis_title='Blocks',
            margin={"l":100},
            legend_title="",
            legend=dict(
                yanchor="top",
                y=0.98,
                xanchor="right",
                x=0.99,
                font=dict(
                    family="Courier New, monospace",
                    size=18,  
                    color=BLACK
                ),
            ),
            xaxis = dict(
               gridcolor = BLACK_ALPHA.format(0.5),
           ),
            yaxis = dict(
                gridcolor = BLACK_ALPHA.format(0.5),     
                fixedrange =True
               ),
            font=dict(
                family="Courier New, monospace",
                size=12,  
                color=BLACK
            ),
        height=550,
        transition = {'duration': 10},
        yaxis_range=[0,df.sort_values("slot", ascending=False).iloc[0]["slot"]*1.1],
        #paper_bgcolor='#eee'
        paper_bgcolor='#ffffff',
        plot_bgcolor='#eee'
        )
    fig.update_traces(hovertemplate = '%{x}<br><b>%{y:,.0f} blocks built</b>',textposition='inside', 
                          hoverlabel=dict(font=dict(color='#ffffff')),
                          marker=dict(line=dict(color="#000000", width=2)), 
                          textfont=dict(color="#ffffff"), opacity=0.95)
    fig.layout.sliders[0].pad.t = 120
    fig.layout.sliders[0].currentvalue.prefix = "Time Frame: "    
    fig.layout.sliders[0].currentvalue.font.size = 18
    fig.layout.sliders[0].font.size = 18
    _b = df.groupby("builder")["slot"].max().reset_index().sort_values("slot", ascending=False)["builder"].tolist()
    #return fig
    fig.data[0].x = _b
    for ix, f in enumerate(fig.frames):
        y = []
        for i in _b:
            dd = df[df["builder"] == i]
            dd = dd[dd["timestamp"] == f.name]
            if len(dd["slot"]) > 0:
                #print("-")
                #print(dd["slot"])
                y.append(int(dd["slot"]))
            else:
                y.append(0.000001)
        #print(y)
        f.data[0].x = _b
        f.data[0].y = y
        
    fig.data[0].y = df.groupby("builder")["slot"].max().reset_index().sort_values("slot", ascending=False)["slot"].tolist()

    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 30
    fig.layout.updatemenus[0].pad = {"r": 10, "t": 145}
    #fig["layout"].pop("updatemenus")
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=16))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                     title_font=dict(size=20))
    return fig
create_cumulated_builder_chart = create_cumulated_builder()
if JUPYTER:
    create_cumulated_builder_chart.show()

In [ ]:
def create_cumulated_relay():
    print("start create_cumulated_relay")
    DS = "ethereum-data-nero.eth.4_cumulative_relays"
    query = build_query(" relay, timestamp, slot", DS, "ORDER BY relay, slot DESC")
    df = pd.read_gbq(query)
    df = df[df["timestamp"] != max(df["timestamp"])]    
    
    def clean(relaylabel):
        if relaylabel == "Other/None":
            return 'Vanilla<br>Builders'
        
        if relaylabel == "agnostic Gnosis":
            return "Agnostic<br>Gnosis"
        if relaylabel == "ultrasound":
            return "Ultra Sound"
        return relaylabel[0:10]
    df["relay"] = df["relay"].apply(clean)
    df["relay"] = df["relay"].apply(lambda x: x[0].upper()+x[1:]) 
    
    #df["relay"] = df["relay"].apply(lambda x: x[:12])
    
    df = df.groupby(["relay", "timestamp"])["slot"].sum().reset_index().sort_values("timestamp")
    #df = df.rename(columns={'frame': 'time frame'})
    fig = px.histogram(df, x='relay', y='slot',  barmode='group',
                      color_discrete_sequence=[hextorgba(j[1:], 0.8) for j in colors[:2]],
                      pattern_shape_sequence=["/", "/"], animation_frame='timestamp',
                       category_orders = dict(relay=df.sort_values("slot", ascending=False)["relay"].drop_duplicates().tolist())
                      )
    fig.layout.sliders[0]['active'] = len(set(df["timestamp"])) -1
    
    fig.update_layout(
            xaxis_tickangle=45,
            hoverlabel=dict(
               # bgcolor="rgba(255, 255, 255, 0.1)",
                font_size=16,
                font_family="Courier New, monospace"
            ),
            #pap
            title='<span style="font-size: 24px;font-weight:bold;">Blocks per Relay </span>',
            xaxis_title="",
            yaxis_title='Blocks',
            margin={"l":100},
            legend_title="",
            legend=dict(
                yanchor="top",
                y=0.98,
                xanchor="right",
                x=0.99,
                font=dict(
                    family="Courier New, monospace",
                    size=18,  
                    color=BLACK
                ),
            ),
            xaxis = dict(
               gridcolor = BLACK_ALPHA.format(0.5),
           ),
            yaxis = dict(
                #type="log",
                gridcolor = BLACK_ALPHA.format(0.5),     
                #fixedrange =True
               ),
            font=dict(
                family="Courier New, monospace",
                size=12,  
                color=BLACK
            ),
        height=550,
        transition = {'duration': 10},
        yaxis_range=[0,df.sort_values("slot", ascending=False).iloc[0]["slot"]*1.1],
        #paper_bgcolor='#eee',
        plot_bgcolor='#eee'
        )
    fig.update_traces(hovertemplate = '%{x}<br><b>%{y:,.0f} blocks relayed</b>',textposition='inside', 
                          hoverlabel=dict(font=dict(color='#ffffff')),
                          marker=dict(line=dict(color="#000000", width=2)), 
                          textfont=dict(color="#ffffff"), opacity=0.95)
    fig.layout.sliders[0].pad.t = 140
    fig.layout.sliders[0].currentvalue.prefix = "Time Frame: "    
    fig.layout.sliders[0].currentvalue.font.size = 18
    fig.layout.sliders[0].font.size = 18
    _b = df.groupby("relay")["slot"].max().reset_index().sort_values("slot", ascending=False)["relay"].tolist()
    #return fig
    fig.data[0].x = _b
    for ix, f in enumerate(fig.frames):
        y = []
        for i in _b:
            dd = df[df["relay"] == i]
            dd = dd[dd["timestamp"] == f.name]
            if len(dd["slot"]) > 0:
                #print("-")
                #print(dd["slot"])
                y.append(int(dd["slot"]))
            else:
                y.append(0.000001)
        #print(y)
        f.data[0].x = _b
        f.data[0].y = y
        
    fig.data[0].y = df.groupby("relay")["slot"].max().reset_index().sort_values("slot", ascending=False)["slot"].tolist()

    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 30
    fig.layout.updatemenus[0].pad = {"r": 10, "t": 165}
    fig.layout.updatemenus[0]["bgcolor"] = "#ffffff"
    #fig["layout"].pop("updatemenus")
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=18),
                     title_font=dict(size=20))
    return fig
create_cumulated_relay_chart = create_cumulated_relay()
if JUPYTER:
    create_cumulated_relay_chart.show()

In [ ]:
def create_cumulated_validator():
    print("start create_cumulated_validator")
    DS = "ethereum-data-nero.eth.4_cumulative_validators"
    query = build_query(" validator, timestamp, slot", DS, "ORDER BY validator, slot DESC")
    df = pd.read_gbq(query)
    df = df[df["timestamp"] != max(df["timestamp"])]    
    
    def clean(validatorlabel):
        if validatorlabel == "Other/None":
            return 'Vanilla<br>Builders'
        
        if validatorlabel == "agnostic Gnosis":
            return "Agnostic<br>Gnosis"
        return validatorlabel[0:10]
    df["validator"] = df["validator"].apply(clean)
    df["validator"] = df["validator"].apply(lambda x: x[0].upper()+x[1:]) 
    
    #df["validator"] = df["validator"].apply(lambda x: x[:12])
    
    df = df.groupby(["validator", "timestamp"])["slot"].sum().reset_index().sort_values("timestamp")
    #df = df.rename(columns={'frame': 'time frame'})
    fig = px.histogram(df, x='validator', y='slot',  barmode='group',
                      color_discrete_sequence=[hextorgba(colors[0][1:],0.8)],
                      pattern_shape_sequence=["/", "/"], animation_frame='timestamp',
                      category_orders = dict(validator=df.sort_values("slot", ascending=False)["validator"].drop_duplicates().tolist())
                      )
    fig.layout.sliders[0]['active'] = len(set(df["timestamp"])) -1
    
    fig.update_layout(
            xaxis_tickangle=45,
            hoverlabel=dict(
               # bgcolor="rgba(255, 255, 255, 0.1)",
                font_size=16,
                font_family="Courier New, monospace"
            ),
            #pap
            title='<span style="font-size: 24px;font-weight:bold;">Blocks per Validator </span>',
            xaxis_title="",
            yaxis_title='Blocks',
            margin={"l":100},
            legend_title="",
            legend=dict(
                yanchor="top",
                y=0.98,
                xanchor="right",
                x=0.99,
                font=dict(
                    family="Courier New, monospace",
                    size=18,  
                    color=BLACK
                ),
            ),
            xaxis = dict(
               gridcolor = BLACK_ALPHA.format(0.5),
           ),
            yaxis = dict(
                #type="log",
                gridcolor = BLACK_ALPHA.format(0.5),     
                #fixedrange =True
               ),
            font=dict(
                family="Courier New, monospace",
                size=12,  
                color=BLACK
            ),
        height=550,
        transition = {'duration': 10},
        yaxis_range=[0,df.sort_values("slot", ascending=False).iloc[0]["slot"]*1.1],
        paper_bgcolor='#eee',
        plot_bgcolor='#ffffff'
        )
    fig.update_traces(hovertemplate = '%{x}<br><b>%{y:,.0f} blocks proposed</b>',textposition='inside', 
                          hoverlabel=dict(font=dict(color='#ffffff')),
                          marker=dict(line=dict(color="#000000", width=2)), 
                          textfont=dict(color="#ffffff"), opacity=0.95)
    fig.layout.sliders[0].pad.t = 120
    fig.layout.sliders[0].currentvalue.prefix = "Time Frame: "    
    fig.layout.sliders[0].currentvalue.font.size = 18
    fig.layout.sliders[0].font.size = 18
    _b = df.groupby("validator")["slot"].max().reset_index().sort_values("slot", ascending=False)["validator"].tolist()
    #return fig
    fig.data[0].x = _b
    for ix, f in enumerate(fig.frames):
        y = []
        for i in _b:
            dd = df[df["validator"] == i]
            dd = dd[dd["timestamp"] == f.name]
            if len(dd["slot"]) > 0:
                #print("-")
                #print(dd["slot"])
                y.append(int(dd["slot"]))
            else:
                y.append(0.000001)
        #print(y)
        f.data[0].x = _b
        f.data[0].y = y
        
    fig.data[0].y = df.groupby("validator")["slot"].max().reset_index().sort_values("slot", ascending=False)["slot"].tolist()

    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 30
    fig.layout.updatemenus[0].pad = {"r": 10, "t": 145}
    fig.layout.updatemenus[0]["bgcolor"] = "#ffffff"
    #fig["layout"].pop("updatemenus")
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=14))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2),tickfont=dict(size=16),
                     title_font=dict(size=20))
    return fig
create_cumulated_validator_chart = create_cumulated_validator()
if JUPYTER:
    create_cumulated_validator_chart.show()

In [ ]:
def create_total_cumulated_mev_chart():
    print("start create_total_cumulated_mev_chart")
    DS = "ethereum-data-nero.eth.1_cumulated_mev"
    query = build_query("timestamp, value", DS, "ORDER BY timestamp ASC")
    df = pd.read_gbq(query)
    df["value"] = df["value"].astype(float)
    time_axis = df["timestamp"].drop_duplicates().values.tolist()[::int(len(df["timestamp"].drop_duplicates())/10)][:-1]+[df["timestamp"].drop_duplicates().iloc[-1]]
    fig = px.area(df, x="timestamp", 
                  y="value",
                  #color=px.colors.qualitative.Plotly
                 )
    
    fig.update_layout(
        xaxis_title=None,
        yaxis_title=None,
        legend_title="Relay Provider",
        legend=None,
        #title_xanchor="left",
        #title_yanchor="auto",
        hovermode="x unified",
        hoverlabel=dict(
            bgcolor="#ffffff",
            font_size=16,
            font_family="Courier New, monospace"
        ),
        showlegend=False,
        font=dict(
                family="Courier New, monospace",
                size=18,
                color=BLACK
        ),
        
        margin={"l":40, "t":80, "r": 40, "b": 30},
        title=dict(
            text='<span style="font-size: 24px;font-weight:bold;">Total MEV distributed</br>through MEV-Boost <span style="font-size:14px;">(in ETH)<br></span></span>',
            xanchor="center",
            x=0.5,
            yanchor="middle",
            y=0.9,
            pad = {"b": 150, "t": 20},
            font=dict(
                family="Courier New, monospace",
                size=18,
                color=BLACK
            )),
        
        #paper_bgcolor='#eee',
        yaxis=dict(fixedrange =True),
        autosize=True, 
        height=400,
        width=500,
        xaxis=dict(
            #tickmode = 'array',
            #   tickvals = time_axis,
            #   ticktext = [i.split(";")[0] for i in time_axis],
                gridcolor = BLACK,
                
            type="date"
        ),
        paper_bgcolor="#ffffff",
        plot_bgcolor=WHITE
        
    )
    
    fig.data[0].line.color = "rgb(255, 153, 62)"
    fig.data[0].fillpattern.shape = "+"    
    fig.update_traces(hovertemplate="<b>%{y:,.0f} ETH</b>",hoverinfo='x+y', mode='lines', line=dict(width=0.8))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=BLACK_ALPHA.format(0.2))
    
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["value"],
        fill=None,
        mode='lines',
        name="",
        line_color=BLACK, text="", showlegend = False)
                 )
    fig.data[1]["hoverinfo"] = "skip"
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    return fig
create_total_cumulated_mev_chart_chart = create_total_cumulated_mev_chart()
if JUPYTER:
    create_total_cumulated_mev_chart_chart.show()

In [ ]:
query = build_query("COUNT(DISTINCT validator) c", 
                    "ethereum-data-nero.eth.mevboost_db",
                    "WHERE DATE(date) between DATE_SUB(current_date(), INTERVAL 15 DAY) and current_date()")
df = pd.read_gbq(query)
nrofvalidators = int(df["c"])

In [ ]:
DS = "ethereum-data-nero.eth.5_builder_mapping"
query = build_query("label, blocks, builder_pubkey", DS, "ORDER BY blocks DESC")
df_mapping = pd.read_gbq(query).dropna()
df_mapping["label"] = df_mapping.apply(lambda x: "" if x["label"] == x["builder_pubkey"] else x["label"], axis=1)
df_mapping["blocks"] = df_mapping["blocks"] .apply(lambda x: "{:,}".format(int(x)))
df_mapping = clean_xml(df_mapping)
df_mapping = df_mapping.replace('<tr style="text-align: right;', '<tr style="text-align: left;font-size:12px;')
df_mapping = df_mapping.replace('<table', '<table id="buildertable" style="font-size:12px;"')
df_mapping = df_mapping.replace("builder_pubkey", "Builder Public Key")
df_mapping = df_mapping.replace("<th>label", "<th>Label")
df_mapping = df_mapping.replace("<th>blocks", "<th>Blocks")
df_mapping = '<p style="text-align:left;padding-left: 1vw;"><span style="font-size: 24px;font-weight:bold;">Builders Overview (last 14 days)</span></p>' + df_mapping

In [ ]:
SLOTS_SINCE_MERGE = LAST_SLOT - 4700013
DS = "ethereum-data-nero.eth.5_validator_mapping"
query = build_query(
    "validator as Validator, total_registered as Validators, blocks as Blocks_since_merge, total_validators", 
    DS, 
    "ORDER BY blocks DESC"
)

df_mapping_v = pd.read_gbq(query).dropna()
df_mapping_v["Validator"] = df_mapping_v["Validator"].apply(
    lambda x: x[0:12]+"..." if x.startswith("0x") else x[0].upper() + x[1:]
)
df_mapping_v["Blocks_since_merge"] = df_mapping_v["Blocks_since_merge"].apply(
    lambda x: "{:,} ({:.2f}%)".format(int(x), int(x)/SLOTS_SINCE_MERGE*100)
)
df_mapping_v["Validators"] = df_mapping_v.apply(
    lambda x:  "{:,} ({:.2f}%)".format(int(x["Validators"]), int(x["Validators"])/x["total_validators"]*100), axis=1
)
df_mapping_v = df_mapping_v.drop("total_validators", axis=1)
df_mapping_v = clean_xml(df_mapping_v)
df_mapping_v = df_mapping_v.replace('<tr style="text-align: right;', '<tr style="text-align: left;font-size:16px;')
df_mapping_v = df_mapping_v.replace('<table', '<table id="buildertable" style="font-size:16px;"')
df_mapping_v = df_mapping_v.replace('Blocks_since_merge', 'Blocks (since merge)')
df_mapping_v = '<p style="text-align:left;padding-left: 1vw;"><span style="font-size: 24px;font-weight:bold;">Validators Overview</span></p>' + df_mapping_v

In [ ]:

#print("Nr of total validators: {:,.0f}".format(nrofvalidators))

DS = "ethereum-data-nero.eth.3_builders_over_time"
query = build_query("Count(DISTINCT builder) nrofbuilders", DS, "WHERE DATE(timestamp) between DATE_SUB(current_date(), INTERVAL 15 DAY) and current_date()")
nrofbuilders = int(pd.read_gbq(query)["nrofbuilders"]) - 1

df_mev = df_relays[df_relays["relay"]!="Other/None"]
df_other = df_relays[df_relays["relay"]=="Other/None"]
dfr_mev = df_mev.groupby("timestamp")["slot"].sum().reset_index().sort_values("timestamp").iloc[-14:]
dfr_other = df_other.groupby("timestamp")["slot"].sum().reset_index().sort_values("timestamp").iloc[-14:]
nr_mev_blocks14d = dfr_mev["slot"].sum()
nr_nonmev_blocks14d = dfr_other["slot"].sum()
avgvalue = "Ξ {:.6f}".format(df_values["percentile_50"].mean())
percentagemevboost = "{:.2f} %".format(nr_mev_blocks14d/(nr_nonmev_blocks14d+nr_mev_blocks14d)*100)

#<style>
#h1 {
#text-align: center;
#</style>

title = "# <p style='text-align: center;margin:0px;'> __MEV-Boost Dashboard__ </p>"
head =  "<div><div><div style ='float:left;color:#0F1419;font-size:18px'>Lastest known slot: {:,.0f} ({})</div>".format(LAST_SLOT, LAST_TS) \
        +'<div style ="float:right;font-size:18px;color:#0F1419">View <a href="./data.html">raw data</a> </div></div>'\
        +'<div><div style ="float:left;font-size:18px;color:#0F1419;clear: left">Built with 🖤 by '\
        +'<a href="https://twitter.com/nero_eth">Toni Wahrstätter</a></div>'\
        +'<div style ="float:right;font-size:18px;color:#0F1419">View Source on <a href="https://github.com/Nerolation/mevboost.pics">Github</a></div></div></div>'
app = dp.App(
    #'<h1 style="text-align:center;font-family:Georgia;font-variant:small-caps;font-size: 70px;color:#0F1419;">MEV-Boost Dashboard</h1>',
    
    dp.Page(title="General", blocks=[
        title,
        head,
        dp.Group(
        dp.BigNumber(heading="Nr. of Relays (last 14 days)", value=nrofrelayers),
        dp.BigNumber(heading="Nr. of Builders (last 14 days)", value=nrofbuilders),
        dp.BigNumber(heading="Nr. of Validators (last 14 days)", value="{:,.0f}".format(nrofvalidators)),
        dp.BigNumber(heading="% MEV-Boost blocks (last 14 days)", value=percentagemevboost),
        dp.BigNumber(heading="Avg MEV Payment (last 14 days)", value=avgvalue),
        dp.BigNumber(heading="Share MEV of Block Value (last 14 days)", value="{:.1f}%".format(mevboostratio*100))
        , columns=3),
        dp.Group(
        create_pie_total_charts_chart,
        dp.Plot(create_total_cumulated_mev_chart_chart),
        columns=2
        ),
        dp.Plot(create_mev_share_chart_chart),
        
        create_potential_mev_chart_chart,
        create_avg_payment_chart_chart,
        create_sankey_chart
    ]),
    dp.Page(title="Relays", blocks=[
        title,
        head,
        create_pie_relay_charts_chart,
        create_total_share_chart_chart,
        create_total_relay_share_per_hour_chart_chart,
        create_bids_relay_share_chart_chart,
        create_avg_proposer_payment_relay_boxes_chart,
        create_avg_proposer_payment_chart,
        create_cumulated_relay_chart,
        #create_sankey_chart
    ]),
    dp.Page(title="Builders", blocks=[
        title,
        head, 
        create_pie_charts_chart,
        create_total_builder_share_chart_chart,
        create_total_builder_share_per_hour_chart_chart,
        create_bids_builder_share_chart_chart,
        create_avg_proposer_payment_builder_boxes_chart,
        create_avg_proposer_payment_per_builder_chart,
        create_cumulated_builder_chart,
        create_builder_bar_chart_chart,
        df_mapping
        #create_sankey_chart
    ]),
    dp.Page(title="Validators", blocks=[
        title,
        head, 
        #" <p style='text-align: center;margin:0px;'>TBD </p>",
        create_pie_validator_charts_chart,
        create_validator_share_chart_chart,
        create_total_validator_share_per_hour_chart_chart,
        create_cumulated_validator_chart,
        create_avg_proposer_payment_validator_boxes_chart,
        create_lido_validator_share_chart_chart,
        create_avg_proposer_payment_per_validator_chart,
        df_mapping_v
        
        
    #    #create_sankey_chart
    ]),
    #layout=dp.PageLayout.SIDE
   )
app.save(path = GITFOLDER + "index.html",
        formatting=dp.AppFormatting(
        bg_color="#EEE", width=dp.AppWidth.MEDIUM, 
        text_alignment=dp.TextAlignment.RIGHT
    ))

In [ ]:
fixedposi = "<style>nav.min-h-screen {position: -webkit-sticky;position: sticky;}</style>"

more_css = '''

    <style>
    
    body {
        max-width: 900px;
        margin-left: auto !important;
        margin-right: auto !important;
        background: #eee;
    }
    @media screen and (min-width: 700px) {
        body {
            max-width: 1000px;
        }
    }
    
    a.p-2 {
        position: sticky;
        top:0%;
        font-size: 1.4rem;
        padding-top: 1.2rem !important;
        padding-bottom: 1.2rem !important;
    }
    
    div.mb-6 {
        position: sticky;
        top: 0;
        z-index: 99999;
        background-color: white;
    }
    #buildertable tr:nth-child(even) {background: #CCC; border: 1px solid #ddd;}
    #buildertable tr:nth-child(odd) {background: #FFF; border: 1px solid #ddd;}
    
    #buildertable {
        overflow-x: scroll;
        border: 3px solid #fff !important;
        border-collapse: collapse;
    }
    
    th {
        background: #CCC;
        color: rgb(15, 20, 25) !important;
        font-size: 16px;
        padding-left: 1vw !important;
    }
    td {
        padding-right: 1vw  !important;
        padding-left: 1vw  !important;
    }
    
    </style>
'''



</script>"""

with open(GITFOLDER+"index.html", "r") as file:
    f = file.read()
OG_STUFF = ' <title>mevboost.pics | MEV-Boost Dashboard</title>\n<meta charset="UTF-8" />\n<meta name="twitter:card" content="summary_large_image">\n<meta name="twitter:site" content="@nero_ETH">\n<meta name="twitter:title" content="MEV-Boost Relay API Dashboard">\n<meta name="twitter:description" content="Selected comparative visualizations on MEV-Boost and Proposer Builder Separation on Ethereum.">\n<meta name="twitter:image" content="https://mevboost.toniwahrstaetter.com/pv.png">\n<meta property="og:title" content=MEV-Boost Relay API Dashboard>\n<meta property="og:site_name" content=mevboost.pics>\n<meta property="og:url" content=mevboost.pics>\n<meta property="og:description" content="Selected comparative visualizations on MEV-Boost and Proposer Builder Separation on Ethereum." >\n<meta property="og:type" content=website>\n<link rel="shortcut icon" href="https://mevboost.toniwahrstaetter.com/ethlogo.png" />\n<meta property="og:image" content=https://mevboost.toniwahrstaetter.com/pv.png>\n<meta name="description" content="Up-to-date comparative visualizations on MEV-Boost and Proposer Builder Separation on Ethereum.">\n<meta name="keywords" content="Ethereum, MEV-Boost, PBS, Dashboard">\n <meta name="author" content="Toni Wahrstätter">'
f = f.replace('<meta charset="UTF-8" />\n', fixedposi+ OG_STUFF+more_css) # + GA
with open(GITFOLDER + "index.html", "w") as file:
    file.write(f)

In [ ]:
log("ending mevboost.pics chart generation")